<h1><center>Лабораторна робота 8.</center></h1>
<h2><center>Реалізація методів онлайн-навчання</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

У цій лабораторній роботі вам пропонується реалізувати два методи та побудувати дві моделі – регресор та класифікатор, з використанням мови програмування Python. Обидві моделі будуть навчені за стохастичним градієнтним спуском (Stochastic Gradient Descent, SGD).

<a class="anchor" id="8"></a>

## Зміст

- [8.1. Лінійна регресія та SGD](#lab-8.1)
- [8.2. Логістична регресія та SGD](#lab-8.2)
- [8.3. SGD для задачі класифікації](#lab-8.3)
- [8.4. Логістична регресія та SGDClassifier у задачі класифікації відгуків до фільмів](#lab-8.4)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\radiu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

# задамо графіки у форматі .svg, щоби вони мали кращу чіткість
%config InlineBackend.figure_format = 'svg' 

# підвищимо розмір графіків за замовчуванням
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 7, 5

#from drawdata import draw_scatter, draw_line, draw_histogram

У [лекційній записці 8](https://github.com/radiukpavlo/intelligent-data-analysis/blob/main/01_lecture-notes/ida_lecture-08_sgd-hashing/ida_lecture-08_sgd-hashing.ipynb) було описано процес навчання регресора за SGD - мінімазація квадратичної функції втрат. Далі спробуємо реалізувати цей підхід.

<a class="anchor" id="lab-8.1"></a>

## <span style="color:blue; font-size:1.2em;">8.1. Лінійна регресія та SGD</span>

[Повернутися до змісту](#lab-8)

Спершу потрібно реалізувати клас `SGDRegressor` з такими характеристиками:

- клас успадковується від `sklearn.base.BaseEstimator`;
- конструктор приймає параметри `eta` – крок градієнтного спуску (за замовчуванням $10^{-3}$) та `n_iter` – кількість проходів за вибіркою (за умовчанням 10);
- також у конструкторі мають створюватися списки `mse_` та `weights_` для відстеження значень середньоквадратичної помилки та вектора ваг за ітераціями градієнтного спуску;
- клас має методи `fit` та `predict`;
- метод `fit` приймає на вхід матрицю `X` та вектор `y` (об'єкти `numpy.array`), додає до матриці `X` зліва стовпець з одиниць, ініціалізує вектор ваг `w` **нулями** і в циклі з числом ітерацій `n_iter` оновлює ваги, а також записує значення середньоквадратичної помилки, що вийшло на даній ітерації (MSE та SE будуть надто великими) і вектор ваги `w` у призначені для цього списки;
- наприкінці метод `fit` створює змінну `w_`, в якій зберігається той вектор ваг, за якого помилка є мінімальною;
- метод `fit` повинен повертати поточний екземпляр класу `SGDRegressor`, тобто `self`;
- метод `predict` приймає матрицю `X`, додає до неї зліва стовпець з одиниць і повертає вектор прогнозів моделі, використовуючи створений методом `fit` вектор ваг `w_`.

In [3]:
# Можлива структура класу SGDRegressor
"""
class SGDRegressor(BaseEstimator):
    
    def __init__(self, eta=1e-3, n_iter=10):
        # Ваш код тут
        
    def fit(self, X, y):
        X = # Ваш код тут
        
        w = np.zeros(X.shape[1])
        
        for it in tqdm(range(self.n_iter)):
            for i in range(X.shape[0]):            
                w[0] += 2. * self.eta * (y[i] - w.dot(X[i, :]))
                
                for j in range(1, X.shape[1]):
                    w[j] += # Ваш код тут
                self.weights_.append(w.copy())
                self.mse_.append(mean_squared_error(y, X.dot(w)))
                
        self.w_ = # Ваш код тут
                
        return self
                  
    def predict(self, X):
        # Ваш код тут
        
        return X.dot(self.w_)
"""

'\nclass SGDRegressor(BaseEstimator):\n    \n    def __init__(self, eta=1e-3, n_iter=10):\n        # Ваш код тут\n        \n    def fit(self, X, y):\n        X = # Ваш код тут\n        \n        w = np.zeros(X.shape[1])\n        \n        for it in tqdm(range(self.n_iter)):\n            for i in range(X.shape[0]):            \n                w[0] += 2. * self.eta * (y[i] - w.dot(X[i, :]))\n                \n                for j in range(1, X.shape[1]):\n                    w[j] += # Ваш код тут\n                self.weights_.append(w.copy())\n                self.mse_.append(mean_squared_error(y, X.dot(w)))\n                \n        self.w_ = # Ваш код тут\n                \n        return self\n                  \n    def predict(self, X):\n        # Ваш код тут\n        \n        return X.dot(self.w_)\n'

Перевіримо роботу створеного класу за даними щодо росту та ваги. Прогнозуватимемо зростання (у дюймах) за вагою (у фунтах).

In [4]:
weights_heights_url = 'https://raw.githubusercontent.com/radiukpavlo/intelligent-data-analysis/refs/heads/main/02_assignments/ida_lab-08_implement-sgd/weights_heights.csv'

data_demo = pd.read_csv(weights_heights_url)

In [5]:
plt.scatter(data_demo['Weight'], data_demo['Height']);
plt.xlabel('Weight (pound)')
plt.ylabel('Height (inch)');

In [6]:
X, y = data_demo['Weight'].values, data_demo['Height'].values

Виділимо 70% даних під навчання, 30% – під перевірку та нормалізуємо вибірку за StandardScaler().

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                     test_size=0.3,
                                                     random_state=17)

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape([X_train.shape[0], 1]))
X_valid_scaled = scaler.transform(X_valid.reshape([X_valid.shape[0], 1]))

### <span style="color:red; font-size:1.4em;">Завдання 1</span>

---

**Для всіх варіантів:**

**Навчіть створений вами `SGDRegressor` за вибіркою `(X_train_scaled, y_train)`. Параметри залиште за замовчуванням.**

---

**Варіант 1:**
У цьому завданні вам необхідно створити та навчити модель лінійної регресії, використовуючи алгоритм `SGD` на основі набору даних `weights_heights.csv`. Завдання передбачає реалізацію класу `SGDRegressor` з типовими параметрами, такими як крок градієнтного спуску (`eta`) та кількість ітерацій (`n_iter`). Після навчання моделі побудуйте графік, на якому буде відображено зміну `MSE` залежно від номера ітерації. Також визначте мінімальне значення `MSE`, що було досягнуто, та виведіть відповідний найкращий вектор ваг. Спершу завантажте дані з GitHub за допомогою Pandas, здійсніть попереднє оброблення та нормалізацію даних, розділіть вибірку на навчальну та валідаційну.

*Технічна примітка:* Для завантаження даних застосовується `pd.read_csv` з бібліотеки Pandas, обчислення проводяться за допомогою NumPy, розбиття вибірки здійснюється за допомогою `train_test_split` з модуля `sklearn.model_selection`, а для нормалізації даних використовується `StandardScaler` з `sklearn.preprocessing`. Для побудови графіків використовується Matplotlib (функції `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 2:**
Вам необхідно модифікувати створений клас `SGDRegressor`, додавши можливість адаптивної зміни параметра `eta` протягом ітерацій. Реалізуйте механізм, який зменшує значення `eta` після кожної певної кількості ітерацій або коли `MSE` не покращується. Завантажте дані з файлу `weights_heights.csv`, проведіть їхнє масштабування, розбийте вибірку та навчіть модель з адаптивним кроком. Побудуйте графік, що відображає зміну `MSE` із часом, і проаналізуйте, як адаптивний learning rate впливає на швидкість збіжності моделі та досягнення мінімуму `MSE`.

*Технічна примітка:* Використовуйте Pandas для завантаження даних, `StandardScaler` із `sklearn.preprocessing` для нормалізації, `train_test_split` із `sklearn.model_selection` для розбиття даних, NumPy для математичних операцій, а Matplotlib для побудови графіків. Реалізуйте адаптивну зміну `eta` за допомогою умовних операторів у циклі навчання.

---

**Варіант 3:**
Завдання цього варіанту полягає в створенні регресійної моделі з використанням алгоритму `SGD`, застосовуючи набір даних `weights_heights.csv`. Вам необхідно розробити клас `SGDRegressor` та навчити його, використовуючи стандартні значення параметрів, таких як `eta` і `n_iter`. Після навчання моделі побудуйте графік, який демонструє залежність `MSE` від номера ітерації. Окрім цього, визначте і виведіть оптимальне значення `MSE` та відповідний вектор ваг, що спричинив досягнення цього показника. Аналізуйте, як змінюється `MSE` під час проходження по вибірці, та дослідіть можливості покращення моделі через зміну параметрів.

*Технічна примітка:* Для обробки даних застосовується бібліотека Pandas (`pd.read_csv`), математичні операції виконуються з NumPy, розбиття даних – через `train_test_split` з `sklearn.model_selection`, нормалізація здійснюється за допомогою `StandardScaler` з `sklearn.preprocessing`, а графіки створюються за допомогою Matplotlib (функції `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 4:**
У даному завданні використовуйте набір даних `weights_heights.csv` для побудови лінійної регресійної моделі, що базується на алгоритмі `SGD` (SGD). Ваше завдання – реалізувати клас `SGDRegressor`, використовуючи стандартні параметри (наприклад, `eta=1e-3` та `n_iter=10`), та навчити модель на відповідних даних. Після навчання моделі створіть графік, на якому буде показано, як змінюється середньоквадратична помилка (`MSE`) з кожною ітерацією алгоритму. Обов’язково визначте, на якій ітерації досягнуто мінімального значення `MSE`, та виведіть оптимальний вектор ваг, що забезпечив цей результат. Завдання також включає попереднє оброблення даних, її нормалізацію, а також розбиття даних на навчальну та валідаційну вибірки.

*Технічна примітка:* Для завантаження даних використовується `pd.read_csv` з Pandas, нормалізація здійснюється через `StandardScaler` з `sklearn.preprocessing`, розбиття даних – за допомогою `train_test_split` з `sklearn.model_selection`, обчислення виконуються з NumPy, а для побудови графіків застосовується Matplotlib (методи `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 5:**
Цей варіант завдання спрямований на розширений аналіз процесу навчання регресійної моделі з використанням алгоритму `SGD` на основі даних з `weights_heights.csv`. Вам необхідно реалізувати клас `SGDRegressor` з типовими параметрами та навчити його на навчальній вибірці, після чого побудувати графік, що демонструє, як змінюється `MSE` з кожною ітерацією. Окрім відображення динаміки `MSE`, завдання вимагає визначення мінімального значення помилки та виведення найкращого вектора ваг, який забезпечує оптимальний результат. Додатково, порівняйте результати з моделлю, реалізованою через `LinearRegression` з sklearn, щоб оцінити якість навчання вашої моделі.

*Технічна примітка:* Завантаження даних виконується через `pd.read_csv` з Pandas, нормалізація – за допомогою `StandardScaler` з `sklearn.preprocessing`, розбиття даних – через `train_test_split` з `sklearn.model_selection`, обчислення – з NumPy, а побудова графіків здійснюється за допомогою Matplotlib (методи `plt.plot`, `plt.legend`, `plt.grid`).

---

**Варіант 6:**
Реалізуйте механізм ранньої зупинки (early stopping) під час навчання моделі `SGDRegressor` на основі даних із файлу `weights_heights.csv`. Завдання полягає в тому, щоб зупинити навчання, якщо впродовж заданої кількості ітерацій не спостерігається покращення значення `MSE`. Навчіть модель, відстежуйте динаміку `MSE` та встановіть порогове значення для зупинки процесу. Побудуйте графік, на якому буде відображено зміну `MSE` від номера ітерації, і позначте момент, коли навчання було припинено. Порівняйте результати з варіантом стандартного SGD без ранньої зупинки та проаналізуйте ефективність застосування цієї методики для зменшення часу навчання моделі.

*Технічна примітка:* Використовуйте Pandas для завантаження даних, `StandardScaler` із `sklearn.preprocessing` для нормалізації, `train_test_split` із `sklearn.model_selection` для розбиття даних; для обчислення `MSE` – `mean_squared_error` із `sklearn.metrics`; побудову графіків здійснюйте за допомогою Matplotlib.

---

**Варіант 7:**
Ваше завдання – реалізувати клас `SGDRegressor` зі стандартними параметрами, навчити модель на навчальній вибірці та побудувати графік, який демонструє, як змінюється `MSE` з кожною ітерацією. Особливу увагу зверніть на динаміку зменшення `MSE`, визначте момент, коли помилка досягає свого мінімуму, та виведіть оптимальний вектор ваг, що відповідає цьому показнику. Також експериментуйте з варіаціями параметрів, наприклад, змінюючи значення `eta` та `n_iter`, щоб дослідити їхній вплив на якість навчання. Для цього спершу завантажте дані за допомогою Pandas, проведіть їхнє попереднє оброблення та нормалізацію за допомогою StandardScaler, а потім розбийте дані на навчальну та тестову вибірки. Результати аналізу візуалізуйте через Matplotlib, що допоможе отримати детальну картину процесу навчання.

*Технічна примітка:* Завантаження даних здійснюється через `pd.read_csv` з Pandas, масштабування виконується за допомогою `StandardScaler` з `sklearn.preprocessing`, розбиття вибірки – за допомогою `train_test_split` з `sklearn.model_selection`, математичні обчислення – за допомогою NumPy, а побудова графіків – через Matplotlib (методи `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 8:**
Вам необхідно реалізувати клас `SGDRegressor` зі стандартними параметрами, навчити його на навчальній вибірці, а потім побудувати графік, який відображає зміни `MSE` залежно від номера ітерації. Особлива увага приділяється визначенню найнижчого значення `MSE` та відповідного оптимального вектора ваг, який забезпечує найкращий прогноз. Додатково проведіть аналіз впливу різних початкових умов ініціалізації ваг на результати навчання. Спочатку завантажте дані з GitHub за допомогою Pandas, проведіть їхнє попереднє оброблення, нормалізацію за допомогою StandardScaler, а також розбиття вибірки за допомогою `train_test_split`. Результати навчання моделі візуалізуйте за допомогою Matplotlib, що дасть змогу отримати наочне уявлення про процес оптимізації.

*Технічна примітка:* Використовуйте `pd.read_csv` з Pandas для завантаження даних, `StandardScaler` з `sklearn.preprocessing` для нормалізації, `train_test_split` з `sklearn.model_selection` для розподілу даних, NumPy для математичних операцій, а для побудови графіків – Matplotlib (функції `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 9:**
У цьому завданні вам потрібно модифікувати стандартну реалізацію алгоритму `SGD`, додавши можливість використання міні-батчів замість оброблення окремих зразків. Розробіть клас `SGDRegressor`, який дасть змогу задавати розмір міні-батча як додатковий параметр. Завантажте набір даних `weights_heights.csv`, проведіть попереднє оброблення та нормалізацію, після чого розбийте дані на навчальну та валідаційну вибірки. Навчіть модель із використанням міні-батчів, побудуйте графік зміни `MSE` від номера ітерації, порівняйте результати з варіантом, де використовується стандартний SGD (оброблення за одним зразком). Детально проаналізуйте вплив розміру міні-батча на збіжність моделі та швидкість навчання.

*Технічна примітка:* Для роботи з даними використовуйте Pandas, обчислення – NumPy, нормалізація – `StandardScaler` із `sklearn.preprocessing`, розбиття даних – `train_test_split` із `sklearn.model_selection`; для візуалізації – Matplotlib. Реалізуйте цикл оновлення ваг з урахуванням міні-батчів за допомогою функцій NumPy (наприклад, `np.mean` для обчислення середнього градієнта по батчу).

---

**Варіант 10:**
Завдання полягає в тому, щоб реалізувати клас `SGDRegressor` зі стандартними параметрами, навчити його на навчальній вибірці, а потім побудувати графік, що демонструє зміну `MSE` залежно від номера ітерації. Обов’язково визначте оптимальне значення `MSE` та відповідний вектор ваг, який забезпечує найкращий результат. Додатково проведіть експеримент зі зміною значень параметрів алгоритму (наприклад, кроку `eta`), щоб дослідити їхній вплив на динаміку навчання. Спершу завантажте дані з GitHub за допомогою Pandas, здійсніть їхню нормалізацію за допомогою StandardScaler, розбийте вибірку на навчальну та валідаційну за допомогою `train_test_split`, а всі математичні обчислення виконуйте за допомогою NumPy. Отримані результати візуалізуйте за допомогою Matplotlib, що дасть змогу отримати наочну картину процесу оптимізації моделі.

*Технічна примітка:* Для роботи з даними використовуйте Pandas (`pd.read_csv`), для нормалізації – `StandardScaler` з `sklearn.preprocessing`, розбиття вибірки здійснюється через `train_test_split` з `sklearn.model_selection`, математичні операції проводяться з NumPy, а побудова графіків здійснюється за допомогою Matplotlib (методи `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 11:**
У цьому завданні вам необхідно реалізувати модель регресії за допомогою алгоритму `SGD` на основі набору даних `weights_heights.csv`. Ваше завдання – розробити клас `SGDRegressor`, який крім стандартного оновлення ваг реалізує збереження історії змін параметрів упродовж кожної ітерації. Зверніть увагу на те, що під час навчання потрібно обчислювати `MSE` та зберігати значення ваг на кожній ітерації. Також варто експериментувати з впливом початкової ініціалізації ваг та параметра кроку `eta` на швидкість збіжності. Після навчання побудуйте графік, де на осі абсцис буде позначено номер ітерації, а на осі ординат – значення `MSE`. Наприкінці завдання виведіть оптимальний набір ваг та мінімальне досягнуте значення `MSE`.

*Технічна примітка:* Використовуйте `pd.read_csv` для завантаження даних, `StandardScaler` із `sklearn.preprocessing` для масштабування, `train_test_split` із `sklearn.model_selection` для розбиття даних, математичні операції за допомогою NumPy, а для побудови графіків – Matplotlib (`plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 12:**
Вам потрібно завантажити дані з файлу `weights_heights.csv`, провести їхнє попереднє оброблення та нормалізацію, а потім розбити вибірку на навчальну та валідаційну. Спочатку навчіть модель, яку ви реалізували самостійно, використовуючи алгоритм `SGD` зі стандартними параметрами. Після завершення навчання побудуйте графік зміни `MSE` від номера ітерації. Далі повторіть навчання, використовуючи `LinearRegression` із sklearn, і розрахуйте `MSE` для валідаційної вибірки. Проведіть детальний аналіз отриманих результатів, порівняйте час виконання та якість прогнозів обох моделей.

*Технічна примітка:* Завантаження даних – `pd.read_csv`; нормалізація – `StandardScaler` з `sklearn.preprocessing`; розбиття вибірки – `train_test_split` із `sklearn.model_selection`; обчислення `MSE` – функція `mean_squared_error` з `sklearn.metrics`; для побудови графіків – Matplotlib.

---

**Варіант 13:**
Ваше завдання полягає в тому, щоб реалізувати клас `SGDRegressor` зі стандартними параметрами, навчити його на навчальній вибірці та побудувати графік, що демонструє зміну `MSE` з кожною ітерацією. Особливо зверніть увагу на визначення моменту, коли `MSE` досягає мінімуму, та виведіть відповідний оптимальний вектор ваг. Крім того, дослідіть вплив різних значень параметра кроку `eta` на процес навчання моделі. Початковий етап включає завантаження даних за допомогою Pandas, їхнє оброблення та масштабування через StandardScaler, а також розбиття вибірки за допомогою `train_test_split`. Подальші обчислення виконуються з використанням NumPy, а результати візуалізуються за допомогою Matplotlib, що дасть змогу детально відслідкувати процес оптимізації моделі.

*Технічна примітка:* Для завантаження даних використовуйте `pd.read_csv` з Pandas, нормалізацію здійснюйте за допомогою `StandardScaler` з `sklearn.preprocessing`, розбиття даних – за допомогою `train_test_split` з `sklearn.model_selection`, обчислення виконуються з NumPy, а для побудови графіків застосовуйте Matplotlib (методи `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

---

**Варіант 14:**
У цьому завданні вам треба розширити функціонал класу `SGDRegressor`, додавши можливість одночасного відстеження двох показників: `MSE` та `MAE`. Вам необхідно модифікувати алгоритм оновлення ваг так, щоб зберігати історію обох метрик під час кожної ітерації навчання. Завантажте дані з файлу `weights_heights.csv`, проведіть їхнє масштабування, розбийте вибірку на навчальну та тестову, а потім навчіть модель. Побудуйте два окремих графіки, на одному з яких подайте зміну `MSE`, а на іншому – `MAE` залежно від номера ітерації. Проаналізуйте, чи надає MAE додаткову інформацію щодо стабільності моделі порівняно з `MSE`, та визначте оптимальний набір ваг, за якого обидві метрики мають мінімальні значення.

*Технічна примітка:* Для обчислення MAE використовуйте функцію `mean_absolute_error` з `sklearn.metrics`. Завантаження даних – Pandas; масштабування – `StandardScaler` із `sklearn.preprocessing`; розбиття даних – `train_test_split` із `sklearn.model_selection`; візуалізація – Matplotlib.

---

**Варіант 15:**
У цьому завданні вам потрібно провести попередній аналіз кореляцій між ознаками даних (вага та висота) з файлу `weights_heights.csv` перед побудовою регресійної моделі. Спочатку виконайте аналіз кореляції, використовуючи методи бібліотеки Pandas та побудуйте теплову карту (heatmap) за допомогою Seaborn або Matplotlib для візуалізації залежностей між ознаками. Після цього навчіть модель `SGDRegressor`, реалізовану на основі алгоритму `SGD`, використовуючи нормалізовані дані. Побудуйте графік зміни `MSE` від номера ітерації та визначте, на якій ітерації досягається мінімум `MSE`. Проведіть порівняльний аналіз, щоб встановити, чи впливає попередній аналіз кореляції на налаштування моделі та її ефективність.

*Технічна примітка:* Завантаження даних – `pd.read_csv`; для аналізу кореляції – метод `corr` з Pandas; побудова теплової карти – Seaborn (`sns.heatmap`) або Matplotlib; нормалізація – `StandardScaler` із `sklearn.preprocessing`; розбиття даних – `train_test_split` із `sklearn.model_selection`; візуалізація – Matplotlib.

---

**Варіант 16:**
Ваше завдання – реалізувати клас `SGDRegressor`, налаштувати його зі стандартними параметрами (`eta` та `n_iter`) та навчити модель на навчальній вибірці. Після завершення навчання побудуйте детальний графік, що демонструє, як змінюється середньоквадратична помилка (`MSE`) на кожній ітерації. Особливо зверніть увагу на точку, у якій `MSE` стає мінімальним, і виведіть відповідний вектор ваг, що забезпечує цей результат. Крім того, проведіть аналіз впливу параметрів ініціалізації та кроку навчання на процес оптимізації.

*Технічна примітка:* Використовуйте `pd.read_csv` для завантаження набору даних, `StandardScaler` з `sklearn.preprocessing` для масштабування, `train_test_split` з `sklearn.model_selection` для розбиття даних, а для побудови графіків – Matplotlib (методи `plt.figure`, `plt.plot`, `plt.grid`).

---

**Варіант 17:**
У цьому завданні вам необхідно розширити можливості класу `SGDRegressor`, реалізувавши збереження стану моделі після кожної епохи навчання. Завдання передбачає, що після кожного проходу по навчальній вибірці модель збере інформацію про поточні значення ваг, які згодом будуть збережені у файл (наприклад, у форматі CSV або pickle). Після завершення навчання побудуйте графік зміни `MSE` від номера епохи та вкажіть, на якій епосі досягнуто найкращого результату. Також проведіть аналіз стабільності моделі за різних стартових умов ініціалізації ваг.

*Технічна примітка:* Для збереження файлів використовуйте модуль `pickle` або функції Pandas для збереження DataFrame у CSV. Завантаження даних – `pd.read_csv`; нормалізація – `StandardScaler` із `sklearn.preprocessing`; розбиття даних – `train_test_split` із `sklearn.model_selection`; графіки – Matplotlib.

---

**Варіант 18:**
Цей варіант завдання орієнтований на комплексний аналіз впливу методів масштабування даних на ефективність моделі `SGDRegressor`. Вам необхідно спочатку провести попереднє дослідження різних методів масштабування: StandardScaler та MinMaxScaler, використовуючи дані з файлу `weights_heights.csv`. Після масштабування за допомогою кожного з методів навчіть окрему модель `SGDRegressor` зі стандартними параметрами `SGD`. Побудуйте графіки зміни `MSE` для кожного підходу та порівняйте результати. Проведіть детальний аналіз, який покаже, як різні способи масштабування впливають на збіжність моделі та її прогнозну здатність. Визначте, який метод масштабування дає найкращі результати для даної задачі, та надайте рекомендації щодо його використання.

*Технічна примітка:* Завантаження даних – `pd.read_csv`; для масштабування використовуйте `StandardScaler` та `MinMaxScaler` із `sklearn.preprocessing`; розбиття даних – `train_test_split` із `sklearn.model_selection`; математичні операції – NumPy; побудова графіків – Matplotlib.

---

**Варіант 19:**
У цьому завданні необхідно розширити функціональність алгоритму `SGD` через впровадження L2 регуляризації. Реалізуйте новий клас `SGDRegressor`, додавши до функції втрат член регуляризації, який допоможе запобігти перенавчанню моделі. Завантажте набір даних `weights_heights.csv`, проведіть його попереднє оброблення, нормалізацію та розбиття на вибірки. Навчіть модель із L2 регуляризацією, побудуйте графік зміни `MSE` від ітерації та порівняйте результати з моделлю без регуляризації. Проаналізуйте вплив значення коефіцієнта регуляризації на збіжність та узагальнюючу здатність моделі, визначте оптимальне значення гіперпараметра регуляризації для досягнення найкращого результату.

*Технічна примітка:* Для обрахування регуляризаційного члена використовуйте NumPy; завантаження даних – `pd.read_csv`; нормалізація – `StandardScaler` із `sklearn.preprocessing`; розбиття даних – `train_test_split` із `sklearn.model_selection`; обчислення `MSE` – `mean_squared_error` із `sklearn.metrics`; графіки – Matplotlib.

---

**Варіант 20:**
У цьому завданні вам пропонується зосередитися на аналізі впливу параметрів алгоритму `SGD` при розв’язанні задачі регресії на основі даних з `weights_heights.csv`. Реалізуйте клас `SGDRegressor` з базовими параметрами (`eta` та `n_iter` за замовчуванням) та навчіть модель на навчальній вибірці. Ваше завдання – побудувати графік, що ілюструє зміну `MSE` залежно від номера ітерації, а також визначити оптимальне значення `MSE` і відповідний вектор ваг. Особливу увагу приділіть аналізу впливу варіацій параметра `eta` на динаміку зменшення помилки. Проведіть попереднє оброблення даних, їхнє масштабування та розбиття на вибірки, що допоможе отримати чіткі результати.

*Технічна примітка:* Для роботи з даними застосовується Pandas (`pd.read_csv`), нормалізація здійснюється через `StandardScaler` з `sklearn.preprocessing`, розбиття даних – через `train_test_split` з `sklearn.model_selection`, математичні обчислення виконуються за допомогою NumPy, а побудова графіків – через Matplotlib (функції `plt.plot`, `plt.xlabel`, `plt.ylabel`, `plt.title`).

<a class="anchor" id="lab-8.2"></a>

## <span style="color:blue; font-size:1.2em;">8.2. Логістична регресія та SGD</span>

[Повернутися до змісту](#lab-8)

Тепер навчимо логістичну регресію за допомогою такого ж стохастичного підходу.

Розглянемо задачу класифікації. Нехай $X$ – навчальна вибірка розмірів $\ell \times (d+1)$ (перший стовпець – вектор з одиниць), $y$ – вектор відповідей, $y_i \in \{-1, 1\}$ . Логістична регресія з $L_2$-регуляризацією зводиться до задачі мінімізації так:

$$ C\sum_{i=1}^\ell \log{(1 + e^{-y_iw^Tx_i})} + \frac{1}{2}\sum_{j=1}^d w_j^2 \rightarrow min_w.$$

### <span style="color:red; font-size:1.4em;">Завдання 2</span>

---

**Варіант 1:** Формула з одночасним урахуванням L2-регуляризації та динамічним кроком навчання.
У цьому варіанті потрібно формалізувати оновлення ваг логістичної регресії з урахуванням L2-регуляризації. Нехай маємо цільову функцію:
$$ C \sum_{i=1}^{\ell} \log \bigl(1 + e^{-y_i \mathbf{w}^T \mathbf{x}_i}\bigr) + \frac{\alpha}{2} \sum_{j=1}^{d} w_j^2.$$
Тоді правило оновлення ваг для $j$-го коефіцієнта на кожній ітерації стохастичного градієнтного спуску можна записати так:
$$ w_j \leftarrow w_j + \eta(t)\Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr),$$
де $\eta(t)$ – динамічний (залежний від номера ітерації) крок навчання, наприклад, $\eta(t) = \frac{1}{t+1}$. Необхідно реалізувати функцію `def weights_sgd`, яка прийматиме матрицю ознак `X`, вектор відгуків `y` (зі значеннями +1/-1), а також параметри `C`, `alpha` (для регуляризації) і `eta(t)`. Для перевірки правильності реалізації треба завантажити дані `weights_heights.csv`, використати одну (наприклад, вагу) або кілька ознак (вагу і зріст), зібрані в `X`, а потім обчислити втрати (log_loss) на мініпакетах або окремих прикладах і пересвідчитися, що значення функції втрат із часом зменшується.

*Технічна примітка:*
У цьому варіанті заохочується використання `pandas` для зчитування CSV-файла, `numpy` для векторизованих операцій, а також `matplotlib` чи `seaborn` для побудови графіків динаміки втрат. Можна додатково скористатися `sklearn.metrics.log_loss`, щоб зручно оцінювати помилку класифікації.

---

**Варіант 2:** Формула з урахуванням адаптивного моменту (SGD+Momentum).
Тут пропонується вдосконалити правило оновлення ваг, додавши поняття «імпульсу» (Momentum). Базова формула оновлення ваг у логістичній регресії:
$$ w_j \leftarrow w_j + \eta \Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr).$$
Але тепер вводимо додатковий вектор $v_j$ («швидкість» або імпульс), який також оновлюється:
$$ v_j \leftarrow \beta\,v_j + (1 – \beta)\Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr).$$
$$ w_j \leftarrow w_j + \eta\, v_j.$$
Необхідно створити функцію `weights_sgd` з параметром `beta`, що визначає силу впливу історії градієнта, та протестувати її на даних `weights_heights.csv`, розділивши дані на навчальну та валідаційну вибірки. Водночас варто переконатися, що за належного вибору `eta` і `beta` збіжність ваг швидша, ніж у звичайному SGD.

*Технічна примітка:*
У цьому варіанті можна використати `StandardScaler` або будь-який інший підхід для нормалізації ознак з `weights_heights.csv`, щоби прискорити збіжність. Для систематичного аналізу – створити цикл за різними значеннями `beta` та візуалізувати криву втрат за ітераціями за допомогою `matplotlib`.

---

**Варіант 3:** Формула з Nesterov Accelerated Gradient (NAG).
Замість звичайного імпульсу (Momentum) пропонується реалізувати Nesterov Accelerated Gradient. Правило оновлення ваг у такому разі вимагає попередньо робити крок за імпульсом, а потім обчислювати градієнт у «прогнозованому» положенні:
$$ \mathbf{w}_{temp} = \mathbf{w} + \beta\,\mathbf{v},$$
$$ v_j \leftarrow \beta\,v_j + (1-\beta)\,\nabla_{w_j}\mathcal{L}(\mathbf{w}_{temp}),$$
$$ w_j \leftarrow w_j + \eta\,v_j.$$
Градієнт $\nabla_{w_j}\mathcal{L}$ для логістичної регресії обчислюється як і раніше:
$$ \nabla_{w_j}\mathcal{L}(\mathbf{w}) = -\,C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} + \alpha\, w_j.$$
Завдання – написати функцію `weights_sgd` із параметром `beta`, що керує накопиченням імпульсу, і протестувати її на даних `weights_heights.csv`. Переконайтеся, що модель справді швидше досягає мінімуму логістичної функції втрат порівняно з класичним підходом.

*Технічна примітка:*
Під час реалізації варто звернути увагу на порядок операцій і використання проміжних змінних. Для оцінювання ефективності можна записувати значення log_loss після кожної ітерації. Також можна зберігати вектори ваг, щоби побачити, як вони змінюються.

---

**Варіант 4:** Формула з адаптивним кроком методом AdaGrad.
У цьому сценарії необхідно скористатися адаптивним вибором кроку $\eta$ згідно з ідеєю AdaGrad. Базова формула для оновлення ваг:
$$ w_j \leftarrow w_j + \frac{\eta}{\sqrt{G_j + \varepsilon}} \Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j \Bigr),$$
де $G_j$ – це сума квадратів градієнтів для вага $w_j$ на попередніх ітераціях, а $\varepsilon$ – невелике число для уникнення ділення на нуль. Необхідно створити функцію `weights_sgd`, яка оновлюватиме вектори `w` та `G` на кожному кроці. Виконати навчання на даних `weights_heights.csv` (можна будь-який стовпець використати як ознаку, а інший – як допоміжну змінну для формування двокласової задачі, наприклад, класифікація чоловіків і жінок за зростом/вагою). Порівняти динаміку втрат між фіксованим кроком та адаптивним кроком з AdaGrad.

*Технічна примітка:*
Зчитування даних виконується через `pandas.read_csv`, а для зберігання сум попередніх квадратів градієнтів створюється вектор/масив тієї ж розмірності, що й `w`. Для візуалізації варто застосувати `matplotlib.pyplot.plot`, щоби продемонструвати зміну log_loss за епохами.

---

**Варіант 5:** Формула з алгоритмом RMSProp.
Розширюємо підхід AdaGrad до RMSProp, де замість накопичення всіх квадратів градієнтів використовується їхнє експоненційне згладжування. Формула оновлення ваг:
$$ G_j \leftarrow \rho\,G_j + (1 – \rho)\,\bigl(\nabla_{w_j} \mathcal{L}\bigr)^2,$$
$$ w_j \leftarrow w_j + \frac{\eta}{\sqrt{G_j + \varepsilon}} \nabla_{w_j} \mathcal{L},$$
де $\nabla_{w_j} \mathcal{L}$ – той самий вираз, що й  у логістичній регресії з регуляризацією (змінюється лише знак, залежно від напрямку оновлення). Необхідно створити функцію `weights_sgd` з параметром `rho`, протестувати за набором даних `weights_heights.csv` і порівняти швидкість збіжності з реалізацією без RMSProp. Особливу увагу варто приділити вибору початкових значень $G_j$ та $\varepsilon$.

*Технічна примітка:*
Знадобиться `numpy` для основних обчислень і зручний цикл `for` для проходу за епохами. Для оцінюванняякості можна використати `sklearn.metrics.log_loss` або підрахувати кількість правильних класифікацій за певним порогом. Дані з файлу `weights_heights.csv` варто розділити на навчальну й тестову вибірки через `train_test_split` зі `sklearn.model_selection`.

---

**Варіант 6:** Формула з адаптивною швидкістю навчання Adam.
У цьому завданні додаємо дві змінні: для зберігання першого моменту градієнта (середнє значення) та для другого моменту (середнє квадратів). Формули для Adam наступні:
$$ m_j \leftarrow \beta_1\, m_j + (1 – \beta_1)\,\nabla_{w_j}\mathcal{L},$$
$$ v_j \leftarrow \beta_2\, v_j + (1 – \beta_2)\,\bigl(\nabla_{w_j}\mathcal{L}\bigr)^2,$$
$$ \hat{m}_j = \frac{m_j}{1 – \beta_1^t}, \quad \hat{v}_j = \frac{v_j}{1 – \beta_2^t},$$
$$ w_j \leftarrow w_j + \eta \frac{\hat{m}_j}{\sqrt{\hat{v}_j} + \varepsilon}.$$
Треба створити функцію `weights_sgd`, додавши параметри `beta1`, `beta2` і виконати експерименти на даних `weights_heights.csv`. Оскільки Adam часто показує кращу збіжність, варто перевірити, наскільки швидко знижується log_loss і як на результат впливають початкові значення $\beta_1, \beta_2$ та $\eta$.

*Технічна примітка:*
Для зчитування даних використайте `pandas`, обробку – через `numpy`. Задля наочності варто відстежувати показники log_loss не лише наприкінці кожної епохи, а й упродовж кількох міні-батчів (за потреби можна розбивати вибірку на блоки фіксованого розміру).

---

**Варіант 7:** Оцінка стабільності формули з великою кількістю ітерацій.
Тут потрібно формалізувати базову формулу:
$$ w_j \leftarrow w_j + \eta \Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr),$$
і запустити її на значно більшій кількості епох (наприклад, 100 або навіть 300) на даних `weights_heights.csv`. Завдання – перевірити, чи не призведе велика кількість проходів вибіркою до перенавчання (overfitting). Виконайте відстеження втрат за навчальним та валідаційним наборах і зробіть висновки про стабільність параметрів.

*Технічна примітка:*
Бажано використати `sklearn.model_selection.train_test_split` для розбиття даних, а також `matplotlib` для побудови двох кривих втрат: на навчанні та валідації. Корисно також застосувати `sklearn.preprocessing.StandardScaler`, щоби поліпшити стабільність навчання.

---

**Варіант 8:** Формула з урізанням ваг (Weight Decay / Decoupled Weight Decay).
Завдання полягає в тому, щоб інтегрувати механізм «decoupled weight decay», який дещо відрізняється від класичної L2-регуляризації. Ідея полягає в додаванні кроку зменшення ваг:
$$ w_j \leftarrow w_j – \eta\,\lambda\,w_j,$$
окремо від оновлення ваг на основі градієнта:
$$ w_j \leftarrow w_j + \eta \Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}}\Bigr).$$
Отже, регуляризаційне зменшення ваг відокремлене від градієнта. Необхідно розробити функцію `weights_sgd` із параметром `lambda`, що визначає силу урізання ваг, і перевірити її на задачі класифікації в наборі даних `weights_heights.csv`. Оцініть, як Weight Decay впливає на збіжність порівняно з класичним підходом, де регуляризація є частиною градієнта.

*Технічна примітка:*
Для системного аналізу можна перебрати декілька значень `lambda` (наприклад, 0.001, 0.01, 0.1), вимірюючи log_loss та точність на валідаційному наборі, а результати вивести у вигляді таблиці через `pandas.DataFrame` та `.plot.bar()` чи `.plot.line()`.

---

**Варіант 9:** Формула з обмеженою кількістю ознак та аналізом ваг.
Реалізуйте логістичну регресію за стохастичним градієнтним спуском із правилом оновлення:
$$ w_j \leftarrow w_j + \eta \Bigl(C \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr).$$
Завдання – детально проаналізувати значення кожного коефіцієнта $w_j$, зокрема пояснити, чому змінна «вага» робить більший чи менший внесок у класифікацію, ніж «зріст» (або навпаки). Пропонується взяти набір даних `weights_heights.csv`, залишити лише частину ознак (скажімо, один стовпець для зросту, один – для ваги) та вручну створити бінарну мітку на кшталт «зріст вище за середнє» проти «зріст нижче або дорівнює середньому». Рекомендується експериментувати з різними початковими значеннями $w_j$.

*Технічна примітка:*
Скористайтеся `numpy` для зручної ініціалізації вектора ваг, а також `pandas` для фільтрації даних. Для чисельних експериментів варто проводити кілька повторних прогонів із різним `random_state`, щоби пересвідчитись, що алгоритм стабільно сходиться до подібних ваг.

---

**Варіант 10:** Формула із частковим повторним зчитуванням даних (mini-batch SGD).
Замість послідовного (істинно стохастичного) оновлення ваг або повного проходу (batch), пропонується метод mini-batch. Формула оновлення для кожного $j$ залишається такою ж:
$$ w_j \leftarrow w_j + \eta \Bigl(\frac{C}{m}\sum_{i \in \text{batch}} \frac{y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr),$$
де $m$ – розмір вибраного міні-батчу. Необхідно написати функцію `weights_sgd`, яка на кожному кроці випадково вибиратиме підмножину індексів з `weights_heights.csv`, обчислюватиме градієнт і оновлюватиме ваги. Завдання – експериментально встановити оптимальний розмір міні-батчу (наприклад, 16, 32, 64) і порівняти швидкість навчання та кінцеву точність із варіантом, де міні-батч дорівнює 1 (істинно стохастичний метод) або дорівнює розміру всієї вибірки (batch).

*Технічна примітка:*
Для вибірки підмножини індексів можна використати `numpy.random.permutation`. У процесі навчання варто відслідковувати log_loss чи accuracy на навчальному та валідаційному наборах. У бібліотеці `sklearn.utils` є корисні функції для ітерації по міні-батчах (`sklearn.utils.shuffle` та ін.), які можуть спростити реалізацію.

---

**Варіант 11:** Формула з кроком навчання, що обчислюється на основі норми градієнта.
Реалізуйте формулу для оновлення ваг логістичної регресії, у якій крок навчання $\eta$ залежить від норми градієнта поточного міні-батча. Тобто перед кожним оновленням ваг обчисліть $\|\nabla \mathcal{L}(\mathbf{w})\|$, а потім встановіть $\eta = \frac{c}{1 + \|\nabla \mathcal{L}(\mathbf{w})\|}$, де $c$ – початкова константа. Застосуйте це правило:
$$ w_j \leftarrow w_j + \eta \left(\frac{C\,y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha \, w_j \right),$$
переконавшись, що крок змінюється динамічно в процесі навчання. Використайте файл `weights_heights.csv` для створення задачі бінарної класифікації (наприклад, за ознакою «стать») і перевірте, як швидко знижується логістична втрата на навчальній та валідаційній вибірках.

*Технічна примітка:*
Для обчислення норми градієнта можна скористатися функцією `numpy.linalg.norm`. У ролі гіперпараметрів задайте початкове значення $c$ і регулярізаційний коефіцієнт $\alpha$. Дані з `weights_heights.csv` зручно зчитувати та обробляти через `pandas`, а для поділу на навчальну та валідаційну множини використати `train_test_split` із `sklearn.model_selection`.

---

**Варіант 12:** Застосування модифікованого ступінчастого графіка навчання (Step Decay).
Реалізуйте формулу оновлення ваг логістичної регресії зі звичайним градієнтом:
$$ w_j \leftarrow w_j + \eta(t) \left(\frac{C\,y_i x_{ij}}{1 + e^{\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha \, w_j \right),$$
але нехай $\eta(t)$ змінюється східчасто, тобто зменшується на визначений коефіцієнт $d$ (наприклад, 0.5) після кожних $p$ епох. У такий спосіб на початку навчання ви використовуєте більший крок, а із часом – зменшуєте, щоби краще досягти мінімуму логістичної втрати. Завантажте дані `weights_heights.csv`, побудуйте двокласову задачу (наприклад, розділіть за певним порогом для ознаки «зріст»), реалізуйте цей механізм Step Decay і проаналізуйте зміну якості класифікації з кожним етапом зниження кроку.

*Технічна примітка:*
Рекомендується використати цикл `for` для проходження за кількістю епох, у середині якого перевіряти, чи кратна поточна епоха значенню $p$. Якщо так, то множити поточний крок на коефіцієнт $d$. Порівняйте отриманий результат зі сценарієм, коли крок навчання фіксований.

---

**Варіант 13:** Формула з урахуванням обмеження ваг у заданому діапазоні (Weight Clipping).
Реалізуйте формулу оновлення ваг для логістичної регресії:
$$ w_j \leftarrow w_j + \eta \Bigl(\frac{C\,y_i x_{ij}}{1 + e^{-\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr),$$
але після кожної ітерації потрібно робити «обрізання» (clipping) ваг, щоб вони не виходили за межі $[-k, k]$. Це часто використовується в генеративно-змагальних нейронних мережах, та може бути корисно і для класичних задач, коли потрібно контролювати розмір параметрів. Завантажте `weights_heights.csv`, сформуйте бінарну задачу (наприклад, залежно від певного критерію маси тіла), реалізуйте `weight clipping` у функції `weights_sgd` і вивчіть, як це впливає на збіжність моделі.

*Технічна примітка:*
Операцію обрізання легко виконати через `numpy.clip(w, -k, k, out=w)`. Для візуалізації збіжності рекомендується зберігати значення log_loss після кожної епохи. Перевірте кілька різних значень $k$, щоби визначити, наскільки сильно обмеження ваг уповільнює чи прискорює збіжність.

---

**Варіант 14:** Формула зі стохастичним шумом у градієнті (Stochastic Weight Perturbation).
Реалізуйте правило оновлення ваг:
$$ w_j \leftarrow w_j + \eta \left(\frac{C\,y_i x_{ij}}{1 + e^{-\,y_i\,\mathbf{w}^T \mathbf{x}_i}} \right),$$
але перед кожним оновленням ваги додайте невеликий шум $\delta \sim \mathcal{N}(0, \sigma^2)$ до $w_j$. Це імітує ідею Bayesian Learning, де параметри можуть випадково відхилятися, допомагаючи уникнути попадання у вузькі локальні мінімуми. Для контролю складності моделі можна залишити $\alpha = 0$ або додати регуляризацію окремо. Дані `weights_heights.csv` використовуйте для двокласової задачі, продемонструйте, як шум у вагах впливає на динаміку log_loss і чи можливо покращити здатність моделі до узагальнення.

*Технічна примітка:*
Згенерувати шум просто за допомогою `np.random.normal(0, sigma)`. Рекомендується на кожному кроці створювати новий масив шумів такої ж розмірності, що й вектор ваг, і додавати його: `w += np.random.normal(0, sigma, size=w.shape)`. Для аналізу зручні `matplotlib` і `sklearn.metrics`, аби подивитися на криву втрат і точність на валідаційному наборі.

---

**Варіант 15:** Формула з вивільненням навчання на середніх (Moving Average Weights).
Реалізуйте базову формулу оновлення ваг:
$$ w_j \leftarrow w_j + \eta \Bigl(\frac{C\,y_i x_{ij}}{1 + e^{-\,y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr),$$
але додається ідея зберігати усереднені ваги $\bar{w}_j$ за кілька останніх ітерацій (наприклад, за $m$ кроків). Застосуйте ці усереднені $\bar{w}_j$ як фінальні параметри моделі. Такий підхід близький до Polyak Averaging, де, замість використання тільки останнього вектора $w$, модель використовує середнє значення. Завантажте дані з `weights_heights.csv`, навчіть модель за стохастичним градієнтним спуском, а як результат розгляньте усереднений вектор ваг. Оцініть, чи покращує це стійкість моделі та зменшує розмах втрат.

*Технічна примітка:*
У процесі навчання можна в кожній ітерації зберігати вагу $w^{(t)}$ і поновлювати середнє: $\bar{w} = \frac{t\,\bar{w} + w^{(t)}}{t+1}$. Для перевірки достатньо запускати логістичну регресію на 5–10 епохах. Скористайтеся `pandas` та `numpy`, а графіки будуйте через `matplotlib`.

---

**Варіант 16:** Формула з псевдонескорельованими ознаками (Decorrelated Features).
Реалізуйте формулу оновлення ваг для логістичної регресії за SGD:
$$ w_j \leftarrow w_j + \eta \Bigl(\frac{C\,y_i x_{ij}}{1 + e^{-y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha\, w_j\Bigr).$$
Перед тим як запускати навчання, запропонуйте декореляцію ознак через обчислення кореляційної матриці чи оберненої ковараційної матриці. У функції `weights_sgd` варто також реалізувати можливість внутрішньо застосовувати декореляцію для кожного міні-батча. На наборі даних `weights_heights.csv` обчисліть кореляційну матрицю між «вагою» та «зростом», виконайте трансформацію, щоби зробити їх менш залежними, а потім перевірте, як це впливає на швидкість збіжності й кінцеве значення log_loss.

*Технічна примітка:*
Корисно використати `numpy.corrcoef` або `pandas.DataFrame.corr()`, а далі знайти обернену матрицю через `numpy.linalg.inv`. Після цього можна «білувати» (whitening) ознаки, щоб зменшити їхню кореляцію. Саме навчання логістичної регресії залишається без змін, окрім того, що ви подаєте вже декорельовані ознаки на вхід.

---

**Варіант 17:** Формула зі змінним рівнем регуляризації впродовж навчання.
Реалізуйте оновлення ваг за формулою:
$$ 
w_j \leftarrow w_j + \eta \Bigl(\frac{C\,y_i x_{ij}}{1 + e^{\, -y_i\,\mathbf{w}^T \mathbf{x}_i}} – \alpha(t)\, w_j\Bigr),
$$
де $\alpha(t)$ – змінний коефіцієнт регуляризації, який зменшується з кожною епохою (наприклад, $\alpha(t)=\alpha_0 \cdot \gamma^t$, де $\gamma<1$). Це імітує ситуацію, коли на початку потрібно сильніше штрафувати великі ваги, а під кінець дати можливість моделі тонше підлаштуватися до даних. На базі `weights_heights.csv` сформуйте бінарну класифікацію (скажімо, вагу вище/нижче середнього) та перевірте, за яких значень $\gamma$ та $\alpha_0$ модель показує найкращий баланс між точністю та узагальненням.

*Технічна примітка:*
Реалізація може включати цикл за епохами із поступовим зменшенням $\alpha(t)$. Результати візуалізуйте у вигляді двох кривих: значення log_loss на навчальній та валідаційній множинах. Для обчислення логістичних втрат можна застосувати `sklearn.metrics.log_loss`, а для поділу набору даних – `train_test_split`.

---

**Варіант 18:** Формула з ваговим масштабуванням при збалансуванні класів.
Реалізуйте класичну формулу оновлення ваг:
$$ w_j \leftarrow w_j + \eta \, \omega(y_i) \left(\frac{y_i x_{ij}}{1 + e^{-y_i\,\mathbf{w}^T \mathbf{x}_i}} \right),$$
де $\omega(y_i)$ – це ваговий коефіцієнт класу. Якщо дані `weights_heights.csv` містять дисбаланс (наприклад, більше одного класу, ніж іншого), варто компенсувати це, збільшуючи внесок градієнта від меншого класу і зменшуючи від більшого. Продемонструйте, як додається регуляризація (за потреби) і як змінюється якість класифікації, коли застосовано таке балансування.

*Технічна примітка:*
Скористайтеся `pandas` для підрахунку кількості об’єктів кожного класу і визначення $\omega(+1)$ та $\omega(-1)$. Під час навчання в кожній ітерації перевіряйте мітку $y_i$ і множте градієнт на $\omega(y_i)$. Для вимірювання показників точності (precision, recall, F1) можна використовувати `sklearn.metrics`.

---

**Варіант 19:** Формула з багатокласовим узагальненням (One-vs-Rest SGD).
У цьому варіанті пропонується розширити логістичну регресію зі стохастичним градієнтним спуском на випадок багатокласової класифікації. Можна реалізувати підхід One-vs-Rest, де для кожного класу $k$ створюється окремий вектор ваг $\mathbf{w}^{(k)}$. Правило оновлення для класу $k$ залишається:
$$ w_j^{(k)} \leftarrow w_j^{(k)} + \eta \Bigl(\frac{\mathbf{1}(y_i = k)\, x_{ij} – \sigma_k(\mathbf{x}_i)\, x_{ij}}{N} – \alpha \, w_j^{(k)}\Bigr),$$
де $\sigma_k(\mathbf{x}_i)$ – імовірність того, що об’єкт належить класу $k$ згідно з поточною моделлю, а $N$ – нормувальний коефіцієнт. На прикладі `weights_heights.csv` штучно сформуйте три умовні «класи» (наприклад, розбити зріст на інтервали) та продемонструйте, як вивчити три вектори ваг одночасно.

*Технічна примітка:*
Необхідно визначити функцію `weights_sgd` для кожного класу та об’єднати результати, щоб у кожній ітерації оновлювати ваги для всіх $k$ моделей. Під час передбачення обирається клас із найбільшим $\mathbf{w}^{(k)T}\mathbf{x}$. Для підрахунку точності кожного класу зручно скористатися `sklearn.metrics.classification_report`.

---

**Варіант 20:** Формула з агрегацією результатів кількох незалежних запусків (Ensemble of SGD Models).
Реалізуйте базову формулу оновлення ваг логістичної регресії:
$$ 
w_j^{(r)} \leftarrow w_j^{(r)} + \eta \Bigl(\frac{C\,y_i x_{ij}}{1 + e^{-y_i\,\mathbf{w}^{(r)T}\mathbf{x}_i}} – \alpha\, w_j^{(r)} \Bigr),
$$
де верхній індекс $r$ позначає номер незалежного запуску (репліки). Ідея: виконайте $R$ паралельних чи послідовних ініціалізацій, отримайте $R$ різних наборів ваг і зрештою агрегуйте (усереднюйте) усі $\mathbf{w}^{(r)}$ у фінальний вектор:
$$ w_j^{(\text{ensemble})} = \frac{1}{R}\sum_{r=1}^{R} w_j^{(r)}.$$
Для `weights_heights.csv` створіть бінарну класифікацію, запустіть цей підхід із кількома різними початковими значеннями $w^{(r)}$. Далі перевірте, чи покращується результат на валідаційній множині, коли використовується середнє за кількома моделями, порівняно з однією моделлю.

*Технічна примітка:*
Зручно обгорнути навчання SGD у функцію, яка повертатиме вектор ваг для кожного запуску. Потім можна зберегти всі вектори в список або масив і обчислити їхній середній. Порівняйте log_loss, AUC чи точність окремо для кожного запуску та для ансамблевого варіанта. Для візуалізації підійде звичайний `matplotlib` із побудовою стовпчикових діаграм.

<a class="anchor" id="lab-8.3"></a>

## <span style="color:blue; font-size:1.2em;">8.3. SGD для задачі класифікації</span>

[Повернутися до змісту](#lab-8)

У цій частин лабораторної роботи перед вами стоїть завдання реалізувати клас `SGDClassifier` з такими характеристиками:

- клас успадковується від `sklearn.base.BaseEstimator`;
- конструктор приймає параметри `eta` – крок градієнтного спуску (за замовчуванням $10^{-3}$), `n_iter` – кількість проходів за вибіркою (за умовчанням 10) та C – коефіцієнт регуляризації;
- також у конструкторі мають створюватися списки `loss_` та `weights_` для відстеження значень логістичних втрат та вектора ваг за ітераціями градієнтного спуску;
- клас має методи `fit`, `predict` та `predict_proba`;
- метод `fit` приймає матрицю `X` та вектор `y` (об'єкти `numpy.array`, розглядаємо тільки випадок бінарної класифікації, і значення у векторі `y` можуть бути -1 та 1), додає до матриці `X` ліворуч стовпець з одиниць, ініціалізує вектор ваг `w` **нулями** і в циклі з числом ітерацій `n_iter` оновлює ваги за виведеною вами формулою, а також записує значення log_loss і вектор ваг `w` у призначені для цього списки;
- наприкінці метод `fit` створює змінну `w_`, в якій зберігається той вектор ваг, за якого помилка мінімальна;
- метод `fit` повинен повертати поточний екземпляр класу `SGDClassifier`, тобто `self`;
- метод `predict_proba` приймає матрицю `X`, додає до неї зліва стовпець з одиниць і повертає матрицю прогнозів моделі (таку ж, яку повертають методи `predict_proba` моделей `sklearn`), використовуючи створений методом `fit` вектор ваг `w_`;
- метод `predict` викликає метод `predict_proba` та повертає вектор відповідей: -1, якщо передбачена ймовірність класу 1 менше 0.5 та 1 – в іншому випадку;
- і ще **важливий момент**: щоб уникнути обчислювальних проблем через занадто великі або малі значення під експонентною (overflow & underflow) використовуйте написану нижче функцію `sigma`.

Структура класу `SGDClassifier` має бути такою:

In [9]:
# Можлива структура класу SGDClassifier
"""
class SGDClassifier(BaseEstimator):
    
    def __init__(self, C=1, eta=1e-3, n_iter=10):
        # Ваш код тут
        
    def fit(self, X, y):
        X = # Ваш код тут
        w = np.ones(X.shape[1])
        
        for it in tqdm(range(self.n_iter)):
            for i in range(X.shape[0]):
                w[0] += # Ваш код тут
                for j in range(1, X.shape[1]):
                    w[j] += # Ваш код тут
                
                self.loss_.append(log_loss(y, sigma(X.dot(w))))
                self.weights_.append(w)
    
        self.w_ = # Ваш код тут
        return self
                  
    def predict_proba(self, X):
        X = # Ваш код тут
        p_vec = # Ваш код тут
        return np.hstack([1 - p_vec, p_vec])
    
    def predict(self, X):
        pred_probs = # Ваш код тут
        signs = # Ваш код тут
        # але могли залишитися нулі, якщо значення pred_probs = 0.5
        signs[np.where(signs == 0)] = 1
        return signs
"""

'\nclass SGDClassifier(BaseEstimator):\n    \n    def __init__(self, C=1, eta=1e-3, n_iter=10):\n        # Ваш код тут\n        \n    def fit(self, X, y):\n        X = # Ваш код тут\n        w = np.ones(X.shape[1])\n        \n        for it in tqdm(range(self.n_iter)):\n            for i in range(X.shape[0]):\n                w[0] += # Ваш код тут\n                for j in range(1, X.shape[1]):\n                    w[j] += # Ваш код тут\n                \n                self.loss_.append(log_loss(y, sigma(X.dot(w))))\n                self.weights_.append(w)\n    \n        self.w_ = # Ваш код тут\n        return self\n                  \n    def predict_proba(self, X):\n        X = # Ваш код тут\n        p_vec = # Ваш код тут\n        return np.hstack([1 - p_vec, p_vec])\n    \n    def predict(self, X):\n        pred_probs = # Ваш код тут\n        signs = # Ваш код тут\n        # але могли залишитися нулі, якщо значення pred_probs = 0.5\n        signs[np.where(signs == 0)] = 1\n       

In [10]:
def sigma(z):
    z = z.flatten()
    z[z > 100] = 100
    z[z < -100] = -100
    return 1. / (1 + np.exp(-z))

Перевіримо `SGDClassifier` за даними UCI щодо [раку молочної залози](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)).

In [11]:
from sklearn.datasets import load_breast_cancer

In [12]:
cancer = load_breast_cancer()
# змінимо мітки в змінній 'y' з 0 на -1
X, y = cancer.data, [-1 if i == 0 else 1 for i in cancer.target]

Виділимо 70% даних під навчання, 30% – під перевірку та нормалізуємо вибірку за StandardScaler().

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                     test_size=0.3,
                                                     random_state=17)

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

### <span style="color:red; font-size:1.4em;">Завдання 3</span>

---

**Для всіх варіантів:**

**Реалізуйте клас `SGDClassifier` за структурою та характеристиками, що наведені вище.**

---

**Варіант 1:** Реалізуйте в класі `SGDClassifier` можливість динамічного зменшення кроку навчання (learning rate), використовуючи експоненціальне зменшення за ітераціями. Для цього в методі `fit` після кожного проходу по всій вибірці (або наприкінці кожної епохи) зменшуйте `eta` у декілька разів (наприклад, `eta <- eta * 0.5` чи іншу сталу). Завантажте дані з файлу `weights_heights.csv` та нормалізуйте ознаки (використайте `StandardScaler` із `sklearn.preprocessing`). Навчіть змінений `SGDClassifier` кілька разів, щоби пересвідчитися, що зменшення кроку справді впливає на швидкість збіжності та стабільність логістичних втрат. Виміряйте значення `log_loss` і `ROC AUC` на валідаційній вибірці. Проаналізуйте, наскільки експоненціальне зниження швидкості навчання може вберегти модель від коливань і покращити збіжність.

*Технічна примітка:* Для побудови графіка логістичних втрат використайте `Matplotlib` чи `Seaborn`. Зберігайте проміжні значення `log_loss` у списку `loss_`, щоб відстежувати перебіг навчання. Врахуйте, що такі бібліотеки, як `numpy`, `pandas`, `matplotlib`, `sklearn`, будуть потрібні для оброблення даних та виконання класифікації.

---

**Варіант 2:** Створіть у вашому `SGDClassifier` можливість ранньої зупинки (early stopping), яка буде переривати навчання, коли поліпшення показника `log_loss` або `ROC AUC` на валідаційній вибірці не спостерігатиметься впродовж заданої кількості ітерацій. Використайте дані `weights_heights.csv`, розділивши їх на навчальну та валідаційну частини (наприклад, `train_test_split` зі `sklearn.model_selection`). У циклі епох перевіряйте показник на валідаційній частині: якщо впродовж, скажімо, 3 послідовних ітерацій не бачите поліпшення, припиняйте навчання. Зобразіть на графіку, за скільки епох модель припиняє оновлюватися, і порівняйте з варіантом без ранньої зупинки.

*Технічна примітка:* Необхідно слідкувати за кращим значенням `log_loss` або `ROC AUC` та зберігати відповідний вектор ваг `w_`. Зупинка навчання реалізується через перевірку умови після кожного проходу вибірки (циклу `n_iter`). Для обчислення метрики на валідаційній вибірці використайте метод `predict_proba`.

---

**Варіант 3:** Додайте до `SGDClassifier` можливість використовувати змінний крок навчання за правилом Поляка–Руппеля, де `eta` підлаштовується залежно від попередніх градієнтів. Для цього відстежуйте величину кроку, спираючись на зміни ваг і градієнта на попередній ітерації. Застосуйте цю стратегію до даних `weights_heights.csv`, попередньо виконавши їхню нормалізацію. Оцініть, чи вдалося прискорити навчання порівняно з фіксованим значенням `eta`, та визначте, як це впливає на розмір ваг та кінцеві показники `log_loss`.

*Технічна примітка:* Вам знадобляться допоміжні структури, аби зберігати значення попереднього градієнта. Зверніть увагу, що Python-бібліотеки `numpy` та `pandas` потрібні для роботи з масивами та зчитування CSV, а для аналізу збіжності зручно застосовувати `matplotlib` або `seaborn`.

---

**Варіант 4:** Запровадьте в `SGDClassifier` можливість вибору норми регуляризації: `L1`, `L2` або `ElasticNet`. Нехай параметр `penalty` у конструкторі може приймати значення `'l1'`, `'l2'` або `'elasticnet'`. У випадку `ElasticNet` використайте додатковий коефіцієнт `alpha`, що визначає співвідношення між `L1` та `L2`. Проведіть експерименти на даних `weights_heights.csv`, визначаючи, який тип регуляризації найкраще позначається на точності класифікації та `log_loss`. Подивіться, які ознаки зникають (звертаються в нуль) за `L1` та `ElasticNet`, і які залишаються важливими для моделі.

*Технічна примітка:* Для зручності порівняння використайте однакові значення `eta`, `C` і `n_iter`. Візуалізацію важливості ознак можна виконати, відтворивши стовпчикову діаграму абсолютних значень ваг за допомогою `matplotlib`.

---

**Варіант 5:** Створіть метод `score` у вашому `SGDClassifier`, що обчислює точність (`accuracy`) на заданому наборі даних. Тепер, використовуючи дані з `weights_heights.csv`, виконайте крос-валідацію (5 або 10 фолдів, folds) вашого `SGDClassifier` та отримайте середню точність, а також середні значення `log_loss`. Порівняйте ці результати з аналогічними показниками від `LogisticRegression` зі `sklearn.linear_model`. Проаналізуйте, чи є суттєва відмінність між точностями двох підходів та наскільки стабільно поводиться стохастичний градієнтний спуск у різних фолдах.

*Технічна примітка:* Скористайтеся `cross_val_score` зі `sklearn.model_selection`, передаючи власний клас як естиматор. Уважно визначте методи `fit`, `predict`, `predict_proba`, щоб `cross_val_score` працював коректно. Пам’ятайте додавати стовпець одиниць під час навчання і передбачення.

---

**Варіант 6:** Модифікуйте метод оновлення ваг у `SGDClassifier`, щоб додати момент (momentum). Вага оновлюється не лише пропорційно градієнту поточної ітерації, а й з огляду на попередній крок. У такий спосіб ви можете згладити оновлення і прискорити збіжність. Застосуйте цей підхід до даних `weights_heights.csv`, порівняйте `log_loss` і `ROC AUC` зі стандартним стохастичним градієнтним спуском без моменту. Який метод швидше збігається до прийнятного рівня помилки?

*Технічна примітка:* Щоби рахувати оновлення з моментом, визначте додаткову змінну (наприклад, `v`), яка зберігає попередні зміни ваг, та параметр `beta`, що визначає силу моменту (приміром, 0.9). Для відстеження результатів навчання використайте списки `loss_` та `weights_`, а для візуалізації – `matplotlib`.

---

**Варіант 7:** Додайте можливість міні-батчового (mini-batch) оновлення ваг: розбийте вибірку на невеликі підвибірки (наприклад, по 32 чи 64 об’єкти), а потім обчислюйте середній градієнт на кожному міні-батчі перед оновленням ваг. У межах одного епоху проходьте всі міні-батчі, а тоді зберігайте значення `log_loss`. Використайте `weights_heights.csv`, розділіть дані на навчальну і валідаційну множини, щоби потім порівняти `log_loss` міні-батчового навчання з повністю стохастичним та «повнопакетним» варіантами (batch gradient descent). Визначте, який підхід дає найстабільніші й найшвидші результати.

*Технічна примітка:* Для організації міні-батчів можна або випадково переставляти індекси вибірки через `numpy.random.permutation`, або генерувати їх ітеруванням у циклі. Обчислення середнього градієнта робіть через `np.mean`, застосовуючи його до вибірки рядків міні-батча.

---

**Варіант 8:** Реалізуйте в `SGDClassifier` можливість самостійно визначати функцію активації для обчислення вірогідностей. Наприклад, дайте змогу обирати між стандартною сигмоїдою (`sigmoid`) та гіперболічним тангенсом (`tanh`). Для `tanh` результат імовірності можна перетворювати до відрізка [0,1], наприклад, через `(tanh(w^T x) + 1)/2`. За допомогою цього підходу навчіть модель на даних `weights_heights.csv`, порівняйте `log_loss` у випадках із сигмоїдною та гіперболічною функцією. Визначте, чи є істотна відмінність у швидкості збіжності та кінцевій точності.

*Технічна примітка:* У методі `predict_proba` перевіряйте поточний тип активації: якщо вибрано `'sigmoid'`, застосовуйте `1. / (1. + np.exp(-z))`; якщо `'tanh'`, обчисліть `np.tanh(z)` і зробіть відповідне масштабування. Для графіків і порівняння метрик використовуйте `matplotlib` та `sklearn.metrics`.

---

**Варіант 9:** Додайте до вашого `SGDClassifier` механізм ваги класів (class_weight). Нехай у вибірці `weights_heights.csv` ви штучно зміните співвідношення між двома класами (наприклад, вибравши менше об’єктів одного класу). Тепер проведіть навчання так, аби рідкісний клас мав більшу вагу під час обчислення функції втрат. Для цього додайте в методі `fit` множник до градієнта об’єктів рідкісного класу. Порівняйте результат із випадком, коли ваги класів рівні. Оцініть, як змінюється `ROC AUC` і де модель «помиляється» частіше.

*Технічна примітка:* Зверніть увагу на те, як ви визначаєте рідкісний клас. Можна створити словник кшталту `{1: 1, 0: 5}` і, відштовхуючись від індекса об’єкта, домножувати на відповідний коефіцієнт похідну від логістичної функції втрат. Перевірте, що вказані бібліотеки (зокрема `numpy`, `pandas`) встановлені для читання CSV та формування матриці ознак.

---

**Варіант 10:** Змініть ваш `SGDClassifier` так, щоби в методі `fit` з’явилася опція зважування помилки залежно від її попередньої історії (адаптивна вага помилки). Ідея полягає в тому, щоби підвищувати вагу недавно отриманих помилок, аби модель швидше реагувала на нові дані (аналогія з методами онлайн-навчання для часових рядів). Виконайте навчання на даних `weights_heights.csv`, розділених у хронологічному порядку (уявимо, що ці дані надходять у часі). Порівняйте підсумковий `log_loss` та `accuracy` зі звичайним варіантом. Поясніть, чи може такий підхід бути корисним для потокових даних у реальних застосуваннях.

*Технічна примітка:* Реалізація може полягати у введенні коефіцієнта `gamma`, що поступово збільшує внесок недавніх помилок. Користуйтеся методами бібліотеки `pandas` для сортування і можливого відбору даних у хронологічному порядку, а також класичними функціями зі `sklearn.metrics` для підрахунку метрик.

---

**Варіант 11:** Розширте `SGDClassifier` додаванням механізму автоматичного зменшення параметра регуляризації `C` зі зростанням кількості ітерацій. Тобто на початкових ітераціях тримайте велике значення `C`, щоби модель менше штрафувала великі ваги, а із часом плавно знижуйте `C`, посилюючи регуляризацію. Спробуйте різні формули зменшення (лінійне, експоненціальне). Завантажте дані `weights_heights.csv`, виконайте нормалізацію ознак за допомогою `StandardScaler` з `sklearn.preprocessing` і розділіть вибірку на навчальну та валідаційну. Переконайтеся, що зменшення `C` впродовж епох запобігає перенавчанню. Порівняйте якість класифікації (`log_loss`, `ROC AUC`) та значення ваг, отримані за динамічним і статичним `C`. Визначте, чи вдалося досягти більш узагальнюючої здатності моделі, коли `C` зменшується із часом.

*Технічна примітка:* Налаштування масштабованих ознак виконується стандартною командою:
```python
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
```
Важливо кожен прохід (епоху) зменшувати `C` згідно з обраною формулою. Логіку зменшення реалізуйте прямо в циклі `fit`, зберігаючи ваги в `weights_`.

---

**Варіант 12:** Створіть у своєму `SGDClassifier` опцію `decaying_penalty`, яка в кожній наступній ітерації зменшує вплив регуляризації не лінійно, а за складнішою формулою (наприклад, `C <- C * sqrt(it+1)`, де `it` – це номер епохи). Навчіть цю модифіковану модель на нормалізованих даних `weights_heights.csv`. Виміряйте й порівняйте `log_loss` і `ROC AUC` між базовою реалізацією `SGDClassifier` та новою, перевіривши кілька стратегій згасання регуляризації. Проаналізуйте, яка з формул краще збігається і чи не призводить надто м’яке зниження регуляризації до перенавчання.

*Технічна примітка:* Зверніть увагу, що при оновленні ваг із регуляризацією важливо правильно масштабувати регуляризаційний доданок у градієнті. Використайте `numpy` для векторизованого оновлення ваг.

---

**Варіант 13:** Додайте до `SGDClassifier` можливість виявляти й виключати вибіркові спостереження, що можуть бути «викидами» (outliers). Для цього, перед оновленням ваг на кожному кроці, оцінюйте вплив об’єкта (його градієнт). Якщо він надто великий або суперечить загальному напрямку, проігноруйте цей об’єкт у даному кроці. Розробіть просту евристику, наприклад, якщо `|градієнт| > alpha * середнє_значення_градієнта`, вважати об’єкт викидом. Проведіть експеримент на `weights_heights.csv`, розділивши дані на train/valid. Оцініть, чи зменшує така стратегія загальну `log_loss` та чи підвищує `ROC AUC`.

*Технічна примітка:* Для підрахунку «великого» градієнта зручно вести рухоме середнє або медіану його величин. Можна реалізувати в циклі виведення ключових статистик через `numpy.mean` чи `numpy.median`. Використайте списки `loss_` і `weights_`, щоби прослідкувати динаміку.

---

**Варіант 14:** Реалізуйте механізм адаптивної вибірки прикладів у `SGDClassifier`, коли перші епохи ви переглядаєте випадкові підмножини даних, а на пізніх епохах зосереджуєтеся лише на тих об’єктах, що спричиняли найбільші помилки. Для цього на початку кожного епоху обчисліть, для яких прикладів останній `log_loss` максимальний, і з більшою вірогідністю включайте саме ці приклади в стохастичний крок навчання. Перевірте на `weights_heights.csv`, чи прискорює цей підхід збіжність, і як це впливає на загальні метрики `ROC AUC`, `log_loss` порівняно зі звичайним стохастичним підходом.

*Технічна примітка:* Для контролю ймовірностей відбору прикладів можна використовувати `numpy.random.choice` із параметром `p=…`, утворюючи вектор ваг для вибірки. Збереження «важких» об’єктів (де помилка більша) допомагає швидше зменшувати похибку, але може призвести до локальних мінімумів.

---

**Варіант 15:** Додайте механізм зворотної вибірки (reverse sampling) у ваш `SGDClassifier`, коли на пізніх ітераціях модель спеціально переглядає ті об’єкти, які вже добре класифіковані (похибка мала), щоби перевірити стабільність коректної класифікації. Ідея полягає в тому, аби не «забувати» добре класифіковані приклади. Навчіть модель на `weights_heights.csv`, поміряйте `log_loss` і `accuracy` на валідаційній вибірці. Проаналізуйте, як цей метод впливає на перенавчання: чи виникають ситуації, коли надмірний перегляд «простих» прикладів знижує узагальнюючу здатність.

*Технічна примітка:* Головна відмінність від попереднього варіанта – замість концентруватися на «важких» прикладах, ви іноді переглядаєте «легкі». Можна реалізувати це через параметр `mode='hard'` чи `mode='easy'` у функції вибірки, залежно від стратегії.

---

**Варіант 16:** Спробуйте вбудувати в `SGDClassifier` механізм адаптивного ініціювання ваг через попереднє навчання (warm start). Спочатку виконайте кілька ітерацій навчання звичайної `LogisticRegression` зі `sklearn.linear_model`, візьміть ваги звідти як початкові для свого `SGDClassifier`. Далі робіть стохастичні оновлення. Порівняйте з випадком, коли ви починаєте з нульових ваг. За даними `weights_heights.csv` розгляньте, наскільки змінився час навчання та остаточне значення метрик. Чи може warm start прискорити збіжність?

*Технічна примітка:* Для отримання початкових ваг зверніться до атрибута `coef_` та `intercept_` після навчання `LogisticRegression`. Потім сформуйте вектор `w` потрібної довжини, вставивши `intercept_` у перший компонент. Нормалізацію виконайте через `StandardScaler`, як і раніше.

---

**Варіант 17:** Реалізуйте клас `SGDClassifier` так, щоб він автоматично підміщував випадковий шум до оновлених ваг (stochastic weight perturbation). Такий шум може допомогти моделі уникати локальних мінімумів і плоских точок. Використайте, наприклад, нормальний шум із нульовим середнім і стандартним відхиленням, пропорційним до поточної епохи. Перевірте це на даних `weights_heights.csv`, особливо зверніть увагу на стабільність `ROC AUC`. Чи дійсно додавання шуму допомагає моделі вийти на кращий мінімум?

*Технічна примітка:* Додайте вектор шуму `noise = np.random.normal(0, sigma, size=w.shape)` після кожного оновлення ваг. Параметр `sigma` може бути функцією від номера ітерації, наприклад, `sigma = 1.0 / sqrt(it+1)`. Стежте, щоб шум не був надто великим, інакше модель «розгойдуватиметься».

---

**Варіант 18:** Застосуйте просту гібридну стратегію в `SGDClassifier`, коли перші N епох працюєте з алгоритмом повністю стохастичного оновлення (опрацьовуючи об’єкти один за одним), а решту епох переходите на міні-батчовий підхід (наприклад, батчі розміром 64). Використайте ці дві фази навчання за набором даних `weights_heights.csv`, розділеному на train/valid. Перевірте, чи зможе така комбінація швидше знижувати `log_loss` і чи збільшує вона стабільність ваг на пізніх ітераціях. Зробіть порівняння з повністю стохастичним чи повністю міні-батчовим режимом.

*Технічна примітка:* Реалізацію можна робити, додаючи параметри `stochastic_epochs=N` та `batch_size=64`. Усередині `fit` перевіряйте номер поточної епохи та перемикайте логіку оновлення ваг (по одному об’єкту чи по групі).

---

**Варіант 19:** Забезпечте у своєму `SGDClassifier` режим, у якому ваги оновлюються за формулою з урахуванням імпульсу та згасання розміру кроку (learning rate decay). Наприклад, крок на $і$-й ітерації задавати як `eta_i = eta_0 / (1 + alpha * i)`, а також оновлювати вагу за класичною схемою з моментом `v = beta*v + (1-beta)*grad`, `w = w + eta_i * v`. На наборі `weights_heights.csv` перевірте: які значення `alpha`, `beta` і `eta_0` дають найкращу збіжність за `log_loss` і водночас непоганий `ROC AUC`?

*Технічна примітка:* Пам’ятайте ініціалізувати `v = 0` для всіх компонент ваг. У циклі оновлень підставляйте поточний номер ітерації (ітерації або епохи) у формулу зменшення кроку. Для обчислення метрик використовуйте `sklearn.metrics.log_loss` і `roc_auc_score`.

---

**Варіант 20:** Додайте до свого `SGDClassifier` можливість збирати статистику про середню величину ваг, дисперсію ваг та середній розмір градієнта на кожному епосі. Реалізуйте метод `get_statistics()`, що повертатиме ці три метрики за всі епохи. Запустіть навчання на `weights_heights.csv` (попередньо нормалізуйте ознаки), а потім на валідаційній множині оцініть `log_loss`. Проаналізуйте, як змінюються середня величина, дисперсія ваг та середній градієнт упродовж навчання. Чи дає змогу така статистика краще відстежувати процес збіжності порівняно з одним лише `log_loss`?

*Технічна примітка:* У циклі навчання обчислюйте ці показники за допомогою `numpy.mean` і `numpy.var` для вектора ваг. Середній розмір градієнта оцінюйте, наприклад, через `np.mean(np.abs(grad))`, якщо градієнт зберігається після кожного оновлення. Дані виводьте та візуалізуйте з допомогою `matplotlib` для наочності.

<a class="anchor" id="lab-8.4"></a>

## <span style="color:blue; font-size:1.2em;">8.4. Логістична регресія та SGDClassifier у задачі класифікації відгуків до фільмів</span>

[Повернутися до змісту](#lab-8)

Тепер проведемо тестування логістичної регресії та її версії, яка навчається стохастичним градієнтним спуском, для задачі класифікації відгуків IMDB. Ця задача розглядалася у [лекційній записці №4](https://github.com/radiukpavlo/intelligent-data-analysis/blob/main/01_lecture-notes/ida_lecture-04_linear_models/ida_lecture-04_linear-models_part-2.ipynb) та [лекційній записці №8](https://github.com/radiukpavlo/intelligent-data-analysis/blob/main/01_lecture-notes/ida_lecture-08_online-learning/ida_lecture-08_online-learning.ipynb) цього ж курсу.

Імпортуємо файли та навчимо об'єкт `CountVectorizer` за наявними даними. 

In [15]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

Завантажимо дані [звідси](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) (це пряме посилання на завантаження, а [тут](http://ai.stanford.edu/~amaas/data/sentiment/) опис набору даних). У навчальній і тестовій вибірках по 12500 тисяч хороших і поганих відгуків до фільмів.

```python
from io import BytesIO
import requests
import tarfile

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

def load_imdb_dataset(extract_path=".", overwrite=False):
    #check if existed already
    if os.path.isfile(os.path.join(extract_path, "aclImdb", "README")) and not overwrite:
        print(f"IMDB dataset is already in place.")
        return

    print(f"Downloading the dataset from: {url}")
    response = requests.get(url)

    tar = tarfile.open(mode= "r:gz", fileobj = BytesIO(response.content))

    data = tar.extractall(extract_path)

load_imdb_dataset()
```

In [16]:
PATH_TO_IMDB = "aclImdb"

try:
    reviews_train = load_files(os.path.join(PATH_TO_IMDB, "train"), categories=['pos', 'neg'])
    text_train, y_train = reviews_train.data, reviews_train.target
    reviews_test = load_files(os.path.join(PATH_TO_IMDB, "test"), categories=['pos', 'neg'])
    text_test, y_test = reviews_test.data, reviews_test.target
except FileNotFoundError:
    print(f"The directory {PATH_TO_IMDB} does not exist. Please check the path and try again.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: name 'os' is not defined


In [ ]:
print(f"Number of documents in training data: {len(text_train)}")
print(np.bincount(y_train))
print(f"Number of documents in test data: {len(text_test)}")
print(np.bincount(y_test))

Навчимо за наявними даними `CountVectorizer`, рахуючи [біграми](https://en.wikipedia.org/wiki/Bigram), тобто перейдемо до розрідженого подання даних, де кожному унікальному слову і парі слів у навчальній вибірці відповідає відокремлена ознака. Загалом отримаємо понад 1.5 млн таких ознак.

In [ ]:
%%time
cv = CountVectorizer(ngram_range=(1, 2))
X_train = cv.fit_transform(text_train)
X_test = cv.transform(text_test)

In [ ]:
X_train.shape, X_test.shape

### <span style="color:red; font-size:1.4em;">Завдання 4</span>
 
 ---

**Для всіх варіантів:**

**Навчіть за вибіркою `(X_train, y_train)` логістичну регресію з параметрами за замовчуванням (тільки вкажіть `random_state=17`) та обрахуйте `ROC AUC` за валідаційною вибіркою. Виміряйте час навчання моделі. Дані можна масштабувати, оскільки ознаки – насправді, є лічильниками, і тому вимірюються приблизно в одному діапазоні.**

---

**Варіант 1:** Реалізуйте в модифікованому `SGDClassifier` можливість продовжувати навчання з попередніх ваг (warm start) і випробуйте це на наборі даних `aclImdb_v1.tar.gz`. Для цього після завантаження і початкового оброблення текстових відгуків (наприклад, токенізації та конвертації у вектори через `CountVectorizer` чи `TfidfVectorizer` зі `sklearn.feature_extraction.text`) виберіть частину даних для початкового навчання моделі. Запам’ятайте отримані ваги, а далі продовжуйте навчання на решті навчальних даних без повторної ініціалізації. Виміряйте час на обох етапах та порівняйте зі стандартним методом, коли ваги ініціалізуються нулями. Визначте, чи допомагає warm start швидше досягти прийнятного `ROC AUC`. Окремо виміряйте продуктивність на валідаційній підмножині. Проаналізуйте, чи покращується узагальнююча здатність моделі завдяки зміні схеми навчання.

*Технічна примітка:* Для зручності використайте `pandas` для завантаження даних, `sklearn.model_selection.train_test_split` для створення валідаційної вибірки, а також модуль `time` для оцінювання часу навчання. Для перетворення тексту застосовуйте `TfidfVectorizer` із належною обробкою, наприклад: `TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2))`.

---

**Варіант 2:** Створіть спеціальний режим у `SGDClassifier`, коли швидкість навчання (`eta`) адаптується залежно від довжини кожного тексту в наборі `aclImdb_v1.tar.gz`. Ідея полягає в тому, щоб відгуки з великою кількістю слів (які, ймовірно, несуть більше змісту) давали більший крок оновлення ваг, а короткі відгуки – менший. Наприклад, можна масштабувати `eta` пропорційно до `log(1 + length_of_text)`. Порівняйте отриманий результат із базовою реалізацією без адаптації: виміряйте `ROC AUC` на валідаційних даних, а також простежте динаміку `log_loss` упродовж ітерацій. Чи не надто модель «загрібає» шум від великих текстів?

*Технічна примітка:* Скористайтеся будь-яким із текстових перетворювачів (`CountVectorizer` чи `TfidfVectorizer`), після чого збережіть для кожного зразка довжину тексту. У випадку оновлення ваг враховуйте множник, що залежить від довжини. Для ефективного зберігання ваг та втрат використайте `numpy` і циклічну структуру в методі `fit`.

---

**Варіант 3:** Модифікуйте `SGDClassifier` так, щоб він здійснював періодичний «обхід» ознак (feature shuffling) після кожної епохи. Інакше кажучи, у логістичній моделі під час обчислення градієнтів раз на декілька ітерацій змінюйте порядок, у якому проходите за ознаками в кожному рядку «мішка». Застосуйте цей прийом до масиву ознак, побудованих на `aclImdb_v1.tar.gz`, і дослідіть, чи впливає це на якість (наприклад, на `ROC AUC`). Спочатку векторизуйте тексти, розділіть дані на навчальні та валідаційні вибірки, а тоді порівняйте «обхід ознак» із базовим варіантом без перестановок. Зробіть висновки, чи здатна періодична зміна порядку ознак прискорити збіжність.

*Технічна примітка:* Використайте для перестановки індексів методи з пакету `numpy.random`. Для зчитування та оброблення текстів підійде `scikit-learn`, а для відображення графіків швидкості збіжності – `matplotlib` або `seaborn`.

---

**Варіант 4:** Розширте `SGDClassifier` для одночасного використання декількох стратегій оновлення: міні-батчів і додаткового вагового коефіцієнта для давніших зразків. Ідея полягає в тому, щоби на перших ітераціях використовувати міні-батчі фіксованого розміру (скажімо, 64 приклади), а починаючи з певної епохи, додавати ваговий коефіцієнт до «свіжих» прикладів, ігноруючи ті, які вже кілька разів опрацьовані. Застосуйте таку «гібридну» схему на `aclImdb_v1.tar.gz`, де ознаки отримано через `TfidfVectorizer`. Перевірте, чи зменшує це коливання `log_loss` і чи підвищує `ROC AUC` на валідації. Продемонструйте графік динаміки втрат упродовж епох.

*Технічна примітка:* У Python створіть додаткові параметри, наприклад, `batch_size` і `decay_factor`, реалізувавши всередині `fit` логіку зміни коефіцієнтів. Використайте `numpy` для вибірки підмножин даних, а також `random_state`=17 для відтворюваності експериментів.

---

**Варіант 5:** Запровадьте метод автоматичного підбору оптимального значення `C` (гіперпараметра регуляризації) у вашому `SGDClassifier`, використовуючи крос-валідацію на підмножині даних `aclImdb_v1.tar.gz`. Для цього, наприклад, передбачте функцію `tune_C` у класі, яка почергово випробує низку значень `C` (наприклад, 0.01, 0.1, 1, 10) й обере таке, що забезпечує найбільший середній `ROC AUC` на 3- або 5-fold крос-валідації. Потім запустіть фінальне навчання з обраним `C`. Визначте, як ця стратегія впливає на час навчання. Продемонструйте також найкращий знайдений показник `ROC AUC`.

*Технічна примітка:* Використайте `sklearn.model_selection.KFold` для реалізації крос-валідації, а також стандартні інструменти для токенізації і перетворення тексту (`CountVectorizer` або `TfidfVectorizer`). Пам’ятайте фіксувати `random_state` задля відтворення результатів.

---

**Варіант 6:** Побудуйте у своєму `SGDClassifier` адаптивну функцію втрат, яка крім класичної логістичної компоненти матиме штраф за довгі тексти з надмірною кількістю рідкісних слів (з огляду на частоту кожного слова у всіх відгуках). Введіть додатковий множник до частки втрат, якщо відгук містить значний відсоток рідкісної лексики. Використайте дані `aclImdb_v1.tar.gz`, попередньо обчисліть поширеність слів у всіх позитивних та негативних відгуках, потім у циклі SGD корегуйте втрату за допомогою згаданого множника. Перевірте, як змінюється `ROC AUC` на валідації порівняно зі стандартною логістичною втратою.

*Технічна примітка:* Для обчислення поширеності слів застосуйте словник (mapping) у Python або використайте `Counter` з модуля `collections`. Додатковий множник можна визначати як `1 + alpha * (рідкісні_слова_процент)`, де `alpha` – налаштовуваний коефіцієнт.

---

**Варіант 7:** Додайте до реалізованого раніше `SGDClassifier` зручний метод `partial_fit`, щоби навчання можна було вести потоково, обробляючи порції даних із `aclImdb_v1.tar.gz`. Для перевірки завантажте відгуки пакетами (наприклад, по 2000 зразків), виконуйте `partial_fit` на кожному пакеті, а потім обчислюйте тимчасовий `ROC AUC` на валідаційному наборі. Зробіть графік залежності `ROC AUC` від кількості оброблених пакетів. Поясніть, у яких випадках часткове навчання є доцільнішим за звичайне, та як це дає змогу заощаджувати оперативну пам’ять.

*Технічна примітка:* У `partial_fit` необхідно передбачити логіку оновлення ваг без перезапуску навчання з нуля. Дані можна читати невеликими файлами або за допомогою генератора, щоб імітувати реальний потік. У `sklearn.linear_model.SGDClassifier` наявна аналогічна функціональність для порівняння.

---

**Варіант 8:** Реалізуйте у своєму `SGDClassifier` режим «зважування» не тільки класів, а й ознак. Ідея полягає в тому, щоб для кожної ознаки (наприклад, слова) задати початковий коефіцієнт її важливості (скажімо, навпаки частоті в корпусі текстів), який домножуватиме відповідний елемент градієнта під час оновлення ваг. Застосуйте цей підхід до `aclImdb_v1.tar.gz` і перевірте, чи справді відріджені слова сильніше впливають на остаточне рішення. Порівняйте `ROC AUC` на валідаційній вибірці з базовою реалізацією без зважування ознак. Продемонструйте, які слова отримали найбільші ваги.

*Технічна примітка:* Для зважування ознак у методі `fit` можна завести вектор `feature_importance` такої самої довжини, як і кількість ознак. Під час оновлення ваг цей вектор використовуватиметься як множник для градієнта. Розрахуйте `feature_importance` за принципом: рідкісне слово -> більша початкова важливість.

---

**Варіант 9:** Запровадьте мультимодельну версію `SGDClassifier`, яка зберігатиме кілька моделей паралельно (з різними початковими вагами) і за фінального передбачення усереднюватиме їхні відповіді. Виберіть, наприклад, 5 різних початкових векторів ваг (з різними `random_state`), по черзі навчаючи кожен на `aclImdb_v1.tar.gz`. Потім у випадку виклику `predict_proba` чи `predict` беріть середнє з виходів цих 5 моделей. Перевірте, чи зростає `ROC AUC` порівняно з одиничною моделлю. Оцініть час, що знадобився на навчання, та зробіть висновок, чи виправдовує себе ансамблювання в `SGD` для такого датасету.

*Технічна примітка:* Зберігайте ваги кожної моделі у вигляді списку масивів. Для усереднення ймовірностей (у `predict_proba`) використайте `numpy.mean` з віссю, що відповідає різним моделям. Для класичного передбачення приймайте рішення за порогом 0.5 середньої імовірності.

---

**Варіант 10:** Удоскональте `SGDClassifier` через автоматичне визначення оптимальної кількості епох (`n_iter`) за критерієм стабілізації логістичних втрат на валідаційній вибірці `aclImdb_v1.tar.gz`. Запустіть навчання з невеликим початковим `n_iter`, обчислюйте `log_loss` на валідації після кожної епохи та припиняйте, коли поліпшення менше за визначений поріг (наприклад, 0.0001). Якщо за 5 епох поспіль поліпшення немає, завершіть навчання. Зіставте отримане «авто-зупинене» число епох із фіксованим. Порівняйте `ROC AUC`, час навчання та узагальнюючу здатність моделі.

*Технічна примітка:* У Python можна зберігати найкращий знайдений вектор ваг за кожної епохи, коли помилка на валідації мінімізується. Потрібно організувати цикл, усередині якого для кожної епохи виконується повний прохід даних стохастичним методом, а потім перевіряється зміна `log_loss`. Скористайтеся `time` для заміру часу.

---

**Варіант 11:** Удоскональте ваш `SGDClassifier`, додавши стратегію вибіркових перезапусків (restarts). Зокрема, реалізуйте параметр `restarts=R`, який після кожних кількох епох скидає ваги до попереднього «найуспішнішого» стану, коли модель демонструвала найкращий `ROC AUC` на валідаційній вибірці. Тобто, якщо впродовж певної кількості ітерацій результат не покращується, ви повертаєтеся до кращого вектора ваг і продовжуєте навчання, але з дещо меншою швидкістю (`eta`). Для цього завантажте `aclImdb_v1.tar.gz`, підготуйте дані через `CountVectorizer` або `TfidfVectorizer`, та далі виконуйте навчання в декілька періодів. Виміряйте час навчання, перевірте `ROC AUC` і порівняйте з результатами «без перезапусків». Проаналізуйте, чи допомагає така стратегія уникнути залипання в поганих локальних мінімумах або точках плато. Якщо час навчання надто зростає, поміркуйте про меншу кількість періодів або обмежену кількість потенційних повернень.

*Технічна примітка:* Скористайтеся `numpy` для копіювання найкращих знайдених ваг. Для вимірювання часу підійде модуль `time`. Власне зниження швидкості (`eta`) можна реалізувати як `eta <- eta * 0.5` у випадку повернення до попереднього вектора ваг.

---

**Варіант 12:** Реалізуйте у своєму `SGDClassifier` спеціальний режим для раннього виявлення «зриву» градієнта (gradient explosion). Якщо на будь-якій ітерації розмір градієнта перевищує визначений поріг (наприклад, вище від 10-кратного середнього значення за минулі кроки), виконайте коротку процедуру «відновлення». Скажімо, зменшіть швидкість навчання `eta` у декілька разів та скиньте вектор ваг до останнього стабільного стану. Потім продовжуйте навчання стандартно. Перевірте таку модифікацію на наборі `aclImdb_v1.tar.gz`, порівнявши з базовим варіантом без контролю «зривів». Оцініть, наскільки покращується чи погіршується `ROC AUC` на валідації і чи не надто уповільнюється навчання через часті «відкоти».

*Технічна примітка:* Для обчислення середнього значення градієнта можна тримати рухоме вікно (rolling window) розміру N і використовувати `numpy.mean`. Щоби визначити стан ваг як «стабільний», зберігайте копію ваг щоразу, коли обчислюєте `log_loss` чи іншу метрику.

---

**Варіант 13:** Додайте в `SGDClassifier` можливість адаптивно перемикатися між різними функціями втрат: логістична втрата (log loss) для перших епох і hinge-втрата (як у SVM) для подальших. Ідея полягає в тому, аби на початку модель поводилася як логістична регресія, а коли ваги вже стали «непоганими», перемкнутися на жорсткіше покарання помилок, притаманне hinge-втраті. На наборі `aclImdb_v1.tar.gz` протестуйте, чи встигає модель так побудувати гарне розділення між позитивними й негативними відгуками. Порівняйте `ROC AUC` на валідаційних даних зі стандартним підходом, коли використовується лише логістична втрата. Визначте, за яким критерієм вирішувати, коли переключатися – наприклад, після фіксованої кількості епох або за змінами `log_loss`.

*Технічна примітка:* Для реалізації hinge-втрати можна скористатися формулою: `loss_i = max(0, 1 – y_i * (w · x_i))`. Зверніть увагу на масштабування `y_i ∈ {-1, 1}`. Адаптивне перемикання втрати робіть усередині циклу `fit`.

---

**Варіант 14:** Запровадьте у вашому `SGDClassifier` концепцію вагованої багатокласовості (multilabel adaptation). Нехай кожен відгук із `aclImdb_v1.tar.gz` може належати до одного чи кількох класів водночас (у цьому разі завдання стає багатозначним, не лише позитив/негатив). Для імітації такого сценарію додатково згенеруйте третій «клас» (приміром, відгуки з певним піднабором слів, що може свідчити про жанр). Змініть алгоритм оновлення ваг, аби підтримувати декілька виходів і обчислювати сумарну втрату по всіх виходах. Перевірте, як це працює на штучно розширеному датасеті, вимірявши `ROC AUC` окремо для кожної категорії та порівняйте з двокласовим підходом.

*Технічна примітка:* Для багатокласовості зручно представити ваги як двовимірний масив (кількість виходів × кількість ознак). Під час оновлення враховуйте логістичну втрату для кожного виходу окремо, а потім підсумовуйте градієнти. Використовуйте функції з `sklearn.metrics` (наприклад, `roc_auc_score`) із параметром `average=None`.

---

**Варіант 15:** Розширте `SGDClassifier` для класифікації з підсиленням ваг помилкових передбачень. Ідея полягає в тому, щоб на кожній ітерації автоматично збільшувати ваги тих зразків із `aclImdb_v1.tar.gz`, за якими поточна модель помилилася, і навпаки зменшувати ваги зразків, які вже класифікуються безпомилково. Цей підхід схожий на AdaBoost, проте ви реалізуєте його безпосередньо в `SGD`. Виміряйте, наскільки швидко модель «вчиться» на раніше помилкових прикладах, і перевірте, чи не погіршується узагальнення. Порівняйте час навчання та `ROC AUC` зі звичайним підходом.

*Технічна примітка:* Для підтримування ваг на рівні зразка можна завести масив `sample_weights`, який за оновлення ваг множитиме відповідний градієнт. Щоби перевіряти «помилковість» передбачень, використовуйте метод `predict_proba` чи `predict`, а потім порівнюйте з `y_train`.

---

**Варіант 16:** Запровадьте в `SGDClassifier` опціональну ітерацію безпосередньо над сирими текстами, застосовуючи `HashingVectorizer` на льоту. Тобто в методі `fit` передбачте, що на вхід подаються необроблені рядки з файлу `aclImdb_v1.tar.gz`, а ви всередині робите векторизацію за допомогою `HashingVectorizer`, уникаючи зберігання всього словника в пам’яті. Це може бути корисно для дуже великих текстових даних. Перевірте, наскільки це прискорює чи сповільнює навчання порівняно з `TfidfVectorizer`. Оцініть, чи вистачає точності векторизації, аби досягти порівнянного `ROC AUC`.

*Технічна примітка:* У `sklearn.feature_extraction.text` є клас `HashingVectorizer`, який перетворює текст у хешовані ознаки, не зберігаючи безпосередньо всіх слів. Налаштуйте параметри `n_features` (наприклад, 2**18) і перевірте, чи вистачить цього для точної класифікації.

---

**Варіант 17:** Реалізуйте у `SGDClassifier` детектування пасивних ознак (inactive features). Ідея полягає в тому, щоби під час навчання відстежувати, чи певна ознака (слово) упродовж декількох епох майже не оновлює ваги (градієнт близький до нуля). Якщо ознака надто довго залишається «неактивною», можна її ігнорувати в наступних кроках, що пришвидшить навчання. На наборі `aclImdb_v1.tar.gz` виділіть такі «пасивні» ознаки й заміряйте, скільки часу вдалося заощадити. Порівняйте `ROC AUC` і підрахуйте, чи не надто втратилася якість через відкидання частини ознак.

*Технічна примітка:* Щоб визначати «пасивність», зручно в циклі зберігати сумарний модуль градієнта для кожної ознаки. Якщо за кілька ітерацій цей показник нижче певного порога, вилучайте ознаку з розгляду (наприклад, виключайте її з матриці). Знадобляться `numpy`-вектори та операції маскування.

---

**Варіант 18:** Урізноманітніть ваше рішення, навчаючи паралельно декілька `SGDClassifier`, де кожен модельний екземпляр відповідає за певний бінарний розріз датасету `aclImdb_v1.tar.gz` (наприклад, тексти з певного діапазону довжин, чи тексти зі значною кількістю знакомісць тощо). Після цього «об’єднайте» їх у фінальному рішенні, середньозважуючи ймовірності. Отже, ви отримаєте щось на зразок розподіленого чи модульного навчання: різні класифікатори спеціалізуються на певних підмножинах. Виміряйте, як зростає час і чи помітно виграє `ROC AUC` від такої стратегії. Поясніть, у яких випадках розподіл датасету може давати переваги.

*Технічна примітка:* Для організації паралельного навчання можна використати `joblib.Parallel` чи аналогічні засоби. Розподіл здійснюйте в Python, створюючи фільтри для індексів рядків. Під час передбачення (у `predict_proba`) усереднюйте ймовірності всіх класифікаторів.

---

**Варіант 19:** Удоскональте `SGDClassifier`, надавши можливість вимірювати та зберігати проміжне значення F1-метрики після кожної обробленої порції рядків із `aclImdb_v1.tar.gz`. Після навчання побудуйте графік, де по осі X кількість оброблених прикладів, а по осі Y – проміжний F1. Порівняйте темп зростання F1 з динамікою `ROC AUC`. Поясніть, чому інколи F1 реагує на оновлення ваг інакше, ніж `ROC AUC`. Визначте кількість прикладів, після якої вже спостерігається плато F1, і чи варто продовжувати навчання.

*Технічна примітка:* Використайте `sklearn.metrics.f1_score`, обов’язково звертаючи увагу на параметр `average='binary'` (або інший, якщо кілька класів). Для побудови графіків можна зберігати F1 в список, оновлювати його після кожних k оброблених зразків і врешті викликати `plt.plot`.

---

**Варіант 20:** Додайте опцію «двобічного проходу» у вашому `SGDClassifier`: при стохастичному оновленні ваг для чергового тексту з `aclImdb_v1.tar.gz` одночасно розглядайте цей самий текст із «віддзеркаленими» ознаками (наприклад, видаліть стоп-слова, або змініть відмінювання), фактично подвоюючи кількість спостережень. Ідея – перевірити, чи стане модель стійкішою до різних написань, орфографічних версій або близьких за змістом варіантів тексту. Оцініть, наскільки це збільшує час навчання і як позначається на метриках (зокрема, на `ROC AUC`). Проаналізуйте, чи отримує виграш саме «двобічний» прохід порівняно з простим збільшенням навчальної вибірки.

*Технічна примітка:* У методі `fit` можна додатково викликати функцію, що «віддзеркалює» текст (наприклад, видаляє певні стоп-слова чи замінює їх на спеціальний токен), і відправляє отриманий результат на оновлення ваг. Слідкуйте, аби сумарна кількість проходів не перевищувала розумних меж часу.

### <span style="color:red; font-size:1.4em;">Завдання 5</span>

---

**Варіант 1:** Розширте функціонал `SGDClassifier`, аби виконувати «останні штрихи» з оптимізації за допомогою методу Ньютона після завершення визначеної кількості ітерацій SGD. Ідея полягає в тому, щоби модель досить швидко сходилася під час перших грубих епох стохастичного градієнтного спуску, а потім «доводила» себе методою Ньютона на зменшеному піднаборі даних або спеціально відібраних «найпроблемніших» прикладах з `aclImdb_v1.tar.gz`. Перевірте, чи вдається в такий спосіб збагнути кращий мінімум функції втрат. Для цього попередньо розділіть дані на навчальну і валідаційну вибірки, токенізуйте тексти за допомогою `TfidfVectorizer` та виконайте кілька експериментів із різною кількістю епох SGD перед переходом до методу Ньютона. Зіставте час навчання та показники `ROC AUC`, щоб зрозуміти, чи виправдане таке ускладнення алгоритму. Опишіть, чи варто застосовувати його для великих обсягів даних.

*Технічна примітка:* Використайте для початку звичайний `SGDClassifier`, додавши в нього додатковий крок з оновлення ваг методом Ньютона (наприклад, виконавши один повний прохід по потрібних прикладах). Застосовуйте `numpy` чи `scipy` для обчислення апроксимації Гесса. Контролюйте `time` для оцінювання швидкодії.

---

**Варіант 2:** Створіть у власній реалізації `SGDClassifier` процедуру диференційованого оновлення ваг за групами ознак. Зокрема, припустимо, що набір даних `aclImdb_v1.tar.gz` розбито на «лексичні» ознаки (наприклад, слова чи біграми) та «емоційні» ознаки (наприклад, слова, які явно відображають настрій). Запровадьте дві різні швидкості навчання: одну для лексичних ознак і іншу для емоційних. Реалізуйте в методі `fit` розпізнавання відповідних індексів ознак і застосування відповідних `eta_lexical` або `eta_emotional`. Перевірте, чи таке розділення швидкості допомагає швидше досягати кращого `ROC AUC` і чи не викликає це переобладнання на «емоційних» словах. Зафіксуйте результати на валідаційному піднаборі.

*Технічна примітка:* Для виокремлення «емоційних» слів можна скласти словник частот емоційних лексем (happy, sad, angry тощо) чи використати готові списки «sentiment words». Вектори сформуйте через `TfidfVectorizer` або `CountVectorizer`, аби ідентифікувати позиції «емоційних» ознак. Далі в самому SGD розділіть вектори ваг і градієнти за цими індексами.

---

**Варіант 3:** Запровадьте в `SGDClassifier` можливість підключення зовнішньої регуляризації у вигляді «dropout»: із певною ймовірністю відключати частину ознак (зводити їхні ваги до нуля) на кроку оновлення. Ідея схожа на dropout у нейронних мережах, але тепер вона реалізується безпосередньо в логістичній регресії. Для набору даних `aclImdb_v1.tar.gz`, після перетворення відгуків у вектори, під час стохастичного оновлення на кожному рядку «скидайте» ваги випадкових ознак із певною невеликою ймовірністю (наприклад, 0.1). Перевірте, як це впливає на `ROC AUC` і чи виникають проблеми з надмірною дестабілізацією ваг. Порівняйте з базовим методом без dropout, щоб з’ясувати, наскільки зменшується перенавчання.

*Технічна примітка:* У Python зручно реалізувати «dropout» через `numpy.random.choice` чи `numpy.random.rand`, визначаючи які індекси ваг «вимикаються» в кожному кроці. Контролюйте, щоб цей ефект не призводив до серйозних коливань, можливо, поступово зменшуючи ймовірність dropout.

---

**Варіант 4:** Підготуйте в `SGDClassifier` підтримку змішаного завдання: частина відгуків з `aclImdb_v1.tar.gz` має бінарні мітки (позитив/негатив), а частина даних отримала замість точної мітки лише «ймовірність позитиву» (наприклад, від 0 до 1). Створіть функцію втрат, що поєднує логістичну втрату для даних із бінарними мітками та квадратичну втрату (MSE) для даних, де є лише ймовірнісне оцінювання. Далі в циклі SGD оновлюйте ваги згідно з такою змішаною функцією втрат. Проаналізуйте, наскільки ефективно ця модель навчається та чи вдається їй покращити `ROC AUC` порівняно зі звичайною бінарною постановкою. Поясніть, чому реальні застосунки можуть мати часткову (неповну) розмітку даних.

*Технічна примітка:* Для зберігання «часткових» відгуків можна завести окремий масив, де `y_probas[i]` ∈ [0, 1], і відповідно налаштовувати вагу в «змішаній» функції втрат. Використовуйте `numpy` для формування двох типів втрат: логістичної і квадратичної. Уважно визначайте, у яких об’єктах використовується одна, а в яких – інша втрата.

---

**Варіант 5:** Модифікуйте свій `SGDClassifier` для автоматичної оптимізації параметра `alpha` в регуляризації типу `ElasticNet`. Тобто, крім ваг `w`, під час навчання підлаштовуйте співвідношення між `L1` і `L2` (наприклад, за підходом Лагранжа чи «dual update»). Застосуйте цей алгоритм до даних `aclImdb_v1.tar.gz`. Вимірюйте, як змінюється це `alpha` упродовж ітерацій, та чи знаходить модель помірну комбінацію `L1`/`L2`, що дає вищий `ROC AUC`. Порівняйте час навчання і стабільність ваг із варіантами, коли `alpha` фіксовано.

*Технічна примітка:* Зазвичай у `SGDClassifier` ми вказуємо `loss='log'` і `penalty='elasticnet'`. Однак, тут вищеописана ідея вимагає динамічного оновлення співвідношення `l1_ratio`. Використайте допоміжну формулу або евристику для коригування `l1_ratio` залежно від змін метрики на валідації. Звертайтеся до `numpy` для обчислень.

---

**Варіант 6:** Додайте до `SGDClassifier` процедуру «історичної інерції»: при оновленні ваг на ітерації враховуйте не лише поточний градієнт, а й «середньозважений» градієнт попередніх ітерацій, але з додатковою вагою для далекого історичного періоду (наприклад, параметр `gamma`). Так ви отримаєте узагальнення між класичним momentum і середнім зважуванням. Перевірте на `aclImdb_v1.tar.gz`, чи відбувається швидше згладження коливань і чи помітно змінюється `ROC AUC` порівняно з базовим стохастичним алгоритмом. Додайте графіки логістичної втрати (log_loss), щоби побачити стабільність збіжності. Оцініть, чи можна знижувати `gamma` із часом.

*Технічна примітка:* Для реалізації інерційного підходу зберігайте рухоме середнє історичних градієнтів. Можна завести змінну `historical_grad`, що оновлюється як `historical_grad = gamma * historical_grad + (1 – gamma) * current_grad`. Потім додайте цей зважений градієнт до формули оновлення ваг.

---

**Варіант 7:** Удоскональте процес навчання `SGDClassifier`, інтегрувавши фільтрацію «неінформативних» ознак під час кожної епохи. Ідея: після кожного проходу по `aclImdb_v1.tar.gz` оцініть дисперсію чи середню вагу деяких ознак. Якщо з’ясується, що певна частка ознак (наприклад, 5 %) має майже нульові ваги або нульову дисперсію градієнтів, то можна тимчасово виключити їх із наступного проходу для швидшого навчання. Наприкінці навчання поверніть їх назад і подивіться, чи зміниться якість (робіть додаткові епохи «доочищення»). Перевірте, наскільки скорочується час кожної епохи та як змінюється `ROC AUC` проти навчання без фільтрації ознак.

*Технічна примітка:* Фільтрацію ознак можна робити, сформувавши маску індексів ознак, що залишаються активними. Використовуйте `numpy` для операцій із матрицями. Наприкінці поверніть ознаки, встановивши їхні ваги на попередній рівень або на 0, якщо вони були справді зайвими.

---

**Варіант 8:** Створіть багаторівневу логістичну регресію у своєму `SGDClassifier`. Це означає, що замість одного вектору ваг `w` ви маєте декілька «шарів» ваг: кожен шар формує проміжне подання (наприклад, через невелику нелінійність), а вихід останнього шару використовує log loss для кінцевого результату. Попри те, що це вже схоже на нейронну мережу з одним прихованим шаром, реалізуйте її за принципом логістичної регресії з можливим додаванням регуляризації. Токенізуйте дані `aclImdb_v1.tar.gz`, а потім перевірте, чи покращує двошарова модель `ROC AUC` порівняно з базовою. Зверніть увагу на те, наскільки підвищується складність і час обчислень.

*Технічна примітка:* Замість однієї матриці ваг у вас будуть дві (або більше): перша перетворює вхідні ознаки на проміжні, а друга робить логістичне передбачення. Кожен шар можна оновлювати окремо, але варто пам’ятати, що для першого шару градієнт передаватиметься через другий (ланцюгове правило). Усе можна реалізувати в межах `fit`, використовуючи `numpy`.

---

**Варіант 9:** Реалізуйте «активне навчання» (active learning) у вашому `SGDClassifier`: припустімо, що спершу лише частина відгуків із `aclImdb_v1.tar.gz` має відому мітку (позитив/негатив). Інші мітки «приховані», але модель може запитати додаткові мітки на етапах, коли визначиться, що якісь тексти справді допоможуть їй покращитися. Зокрема, після певної кількості ітерацій виберіть деякі «найневпевненіші» приклади (де `predict_proba` близьке до 0.5) і прикиньте, що ми отримали їхні реальні мітки. Додайте їх у навчання. Подивіться, чи це пришвидшує зростання `ROC AUC` проти варіанта, коли всі мітки відомі від початку. Поясніть корисність «активного навчання» в реальних задачах.

*Технічна примітка:* У Python можна зберігати «непомічені» зразки окремо, час від часу обчислювати їх імовірності через `predict_proba`, сортувати за «наближеністю до 0.5» і випадковим чином обирати ті, які «розкриваються» (в практичних застосунках це роблять люди-анотатори). Потім викликайте `partial_fit` чи аналог для донавчання.

---

**Варіант 10:** Продемонструйте в `SGDClassifier` стратегію «відхилення» оновлень, коли ваги не оновлюються, якщо зміна втрати надто мала. Тобто, якщо поточний градієнт за конкретним зразком призводить до незначного зменшення логістичної втрати (менше порогового значення), ви пропускаєте цей крок. На наборі даних `aclImdb_v1.tar.gz` проекспериментуйте з різними порогами відхилення (скажімо, 1e-4, 1e-5). Перевірте, наскільки зменшується загальний час навчання, оскільки модель уникає «дріб’язкових» оновлень, та як це відбивається на `ROC AUC`. Поясніть, чи є сенс ігнорувати малі покращення, аби прискорити процес, і в яких випадках це може бути шкідливо.

*Технічна примітка:* Використовуйте `numpy` для обчислення зміни втрати. Спершу оцініть поточну втрату `loss_before`, потім припустіться оновлення ваг і порахуйте нову втрату `loss_after`. Якщо різниця `loss_before – loss_after < threshold`, то скасуйте оновлення (поверніть ваги). Таку перевірку можна робити не на кожному екземплярі, а, наприклад, раз на декілька «мікробатчів».

Я відповім як всесвітньо відомий експерт із методів онлайн-навчання, лауреат Національної премії України імені Тараса Шевченка.

**TL;DR**: Нижче наведено 10 нових варіантів (з 11 до 20) до Завдання 5, кожен описує унікальні способи покращення логістичної регресії та класу `SGDClassifier` на базі датасету `aclImdb_v1.tar.gz`.

---

**Варіант 11:** Запропонуйте модифікацію `SGDClassifier`, що використовує стохастичний градієнтний спуск із додатковим підлаштуванням множника для регуляризації після кожного проходу даних. Спершу встановіть слабшу регуляризацію (наприклад, велике значення `C`), аби модель могла «розгулятися» і виявити корисні патерни, а потім поступово посилюйте регуляризацію, зменшуючи `C`. Це дасть шанс уникнути недонавчання на перших етапах і запобігти перенавчанню на завершальних стадіях. Використайте дані `aclImdb_v1.tar.gz`, перетворивши їх у вектори через `TfidfVectorizer` або `CountVectorizer`, і розділіть вибірку на навчальну та валідаційну частини. Перевірте, чи вдається досягти кращого `ROC AUC`, коли параметр `C` на початку великий, а потім зменшується (наприклад, за експоненціальною формулою), порівняно з фіксованим значенням `C`. Покажіть також, який саме графік зміни логістичних втрат ви отримуєте впродовж епох, і як на ньому відбивається зміна регуляризації.

*Технічна примітка:* Використайте `numpy` для опису динаміки зменшення `C`, наприклад, `C_t = C_0 / (1 + alpha * t)` або `C_t = C_0 * exp(-alpha * t)`, де `t` – номер епохи. Реалізуйте це прямо в циклі `fit`, перед оновленням ваг. Оброблення тексту зручно виконувати через `TfidfVectorizer`.

---

**Варіант 12:** Розширте функціонал `SGDClassifier`, увівши внутрішній буфер для «проблемних» зразків (hard examples buffer). На кожній ітерації, коли ви виявляєте, що відгук із `aclImdb_v1.tar.gz` дає велику похибку (наприклад, `predict_proba` далеко помиляється або показує близький до 0.5 прогноз для відгуку, що є впевнено позитивним/негативним), заносьте цей зразок у буфер. Коли кількість таких прикладів перевищить певний поріг, зробіть окремий «прохід» саме за цим буфером і виконайте додаткове оновлення ваг. Перевірте, чи модель швидше вдосконалюється і, чи зростає загальний `ROC AUC` на валідації, оскільки найскладніші приклади отримують додаткову увагу. Проаналізуйте, чи потрібен у такому випадку ранній stop, аби уникнути надмірного пристосування до складних прикладів.

*Технічна примітка:* У Python можна реалізувати буфер як звичайний список, до якого додаються індекси «складних» зразків. Після кожного звичайного проходу (епохи) перевіряйте, чи буфер не переповнений, і якщо так, обробляйте його окремим міні-батчем. Для порівняння швидкості збіжності зручно будувати криву log_loss упродовж ітерацій.

---

**Варіант 13:** Створіть адаптивну схему «перемішування» (data shuffling) у вашому `SGDClassifier`: за замовчуванням ви перемішуєте індекси даних перед кожним епохом, але цього разу змінюйте розмір «блоків» у порядку проходу. Наприклад, на перших ітераціях обробляйте дані маленькими блоками (міні-батчами), а потім переходьте до повністю випадкової вибірки (і вже зовсім на фінальних епохах – знову до невеликих блоків). Ідея полягає в тому, що модель спершу шукає грубий напрямок градієнта (через невеликі підмножини), а потім узагальнює його випадковим повним перемішуванням і знову дрібно коригує. Завантажте та векторизуйте `aclImdb_v1.tar.gz`, поділіть на train/valid, реалізуйте зазначене перемикання режимів «шахування» даних. Виміряйте, чи впливає така стратегія на `ROC AUC` чи на швидкість збіжності.

*Технічна примітка:* У Python зручно створювати генератори, які постачають індекси з різними стратегіями. Спочатку генеруйте серії індексів маленькими блоками (напр., розміром 32), у середині епох перемішуйте все з `numpy.random.permutation`, а в кінці знову вертайтеся до міні-батчів.

---

**Варіант 14:** Удоскональте `SGDClassifier`, додавши можливість «постепохового» підлаштування ознак за допомогою термінів n-gram. Спершу за векторизації текстів `aclImdb_v1.tar.gz` візьміть лише уніграми, щоби не надто великі були розміри. Навчайте модель декілька епох. Потім, якщо ви бачите, що точність недостатня (наприклад, `ROC AUC` застигло на місці), розширте векторизацію, додавши біграми чи триграми, та продовжте навчання з наявними вагами (warm start). Перевірте, чи це покращує результат, оскільки модель уже «знає» базову динаміку слів, а тепер отримує детальніші ознаки. Поясніть, чи корисний такий підхід для масштабних обсягів даних, де негайне застосування всіх n-gram може бути надмірним.

*Технічна примітка:* Для старту можна використати `CountVectorizer(ngram_range=(1,1))` чи `TfidfVectorizer(ngram_range=(1,1))`, а пізніше додати (1,2) або (1,3). Перетворення зручно робити в декілька етапів, зберігаючи попередній набір ваг і маплячи його на розширену матрицю ознак – хоча формально це трохи складно. Можна також уникнути суворої мапи, ініціалізуючи нові ваги нулями.

---

**Варіант 15:** Додайте у ваш `SGDClassifier` внутрішній «геометричний контроль» якості. На кожній епосі, окрім `log_loss`, вимірюйте «кут» між послідовними оновленнями вектора ваг, тобто, наскільки зміни ваг є узгодженими з попереднім кроком. Якщо цей кут стає занадто гострим або тупим (близький до 90 градусів), це може свідчити про нестабільність у навчанні. На наборі `aclImdb_v1.tar.gz` подивіться, чи зміниться характер збіжності, якщо ви, наприклад, знижуватимете крок навчання `eta` при значній неузгодженості оновлень. Поясніть, у який спосіб це може запобігти «петлянню» алгоритму в районі локальних мінімумів чи плато.

*Технічна примітка:* Для вимірювання кута між векторами ваг `w_t` і `w_{t+1}` можна скористатися формулою з косинусом кута: `cos(theta) = (w_t · w_{t+1}) / (||w_t|| * ||w_{t+1}||)`. Якщо `theta` завеликий, корегуйте `eta`. У Python такі вектори порівняйте через `numpy.dot` і `numpy.linalg.norm`.

---

**Варіант 16:** Реалізуйте в `SGDClassifier` схему «ризик-пріоритету» (risk prioritization). Ідея полягає в тому, щоби при зустрічі дуже негативно помилкового передбачення (наприклад, відгук мав бути позитивним, а ймовірність позитиву вийшла нижчою за 0.1) збільшувати крок оновлення саме для цього прикладу. Для менших помилок застосовуйте стандартний крок. У такий спосіб ви фокусуєтеся на найсильніших програшах. Дані `aclImdb_v1.tar.gz` завантажте, векторизуйте і спробуйте варіювати коефіцієнт посилення для великих помилок. Визначте, чи вдається швидше вийти на високий `ROC AUC`, чи не починає модель надмірно реагувати на окремі «нехарактерні» приклади.

*Технічна примітка:* Для підвищення кроку оновлення можна множити `eta` на деякий чинник (наприклад, 2 чи 5), якщо передбачення було абсолютно «протилежне» реальному. Варто контролювати, чи це не спричинить «зрив» градієнта та надто сильне коливання ваг.

---

**Варіант 17:** Запровадьте у вашому `SGDClassifier` можливість «контрольованого відбору прикладів»: якщо показник `predict_proba` для відгуку з `aclImdb_v1.tar.gz` дуже близький до реальної мітки (наприклад, імовірність позитиву 0.99 при `y=1`), ви можете тимчасово пропустити оновлення ваг за цим прикладом, щоби зекономити обчислення, оскільки він уже «добре» класифікований. Ідея схожа на різні Heuristics in Hard Negative Mining: звертати увагу тільки на ті дані, де модель має проблеми. Перевірте, як зменшується час навчання, чи не втрачає модель від цього здатність до узагальнення і як змінюється `ROC AUC`.

*Технічна примітка:* Організуйте методику перевірки «близькості» передбачення, наприклад, якщо `abs(predict_proba – y) < 0.01`, то пропустити оновлення. Уважайте, що у Python кожен пропуск усе ж вимагає обчислень, тож можна об’єднати це з методом міні-батчів, відкидаючи найменш «цікаві» приклади з розрахунку градієнтів.

---

**Варіант 18:** Запропонуйте «двостадійний» сценарій: спершу тренуйте `SGDClassifier` з невеликою кількістю ознак, відібраних за взаємною інформацією чи `chi2` (тільки найтоповіші слова), отримуючи базову модель. Потім розширте набір ознак (додайте всі слова) та продовжуйте навчання, ініціалізуючи ваги моделлю з першої стадії (warm start). Використайте такий метод на `aclImdb_v1.tar.gz`, розділеному на навчальну та валідаційну вибірки, щоби перевірити, чи прискориться збіжність і, чи покращиться підсумкова `ROC AUC`. Поясніть, у яких випадках цей підхід доцільний: наприклад, коли повна кількість ознак є дуже великою, і ви хочете швидкого старту навчання.

*Технічна примітка:* Для відбору ознак скористайтеся `SelectKBest` із `sklearn.feature_selection`, використавши критерій `chi2` чи `mutual_info_classif`. Спочатку перетворіть дані в меншу кількість ознак, навчіть модель. Потім збережіть ваги й розширте набір ознак, ініціалізувавши параметри SGD тим, що було знайдено.

---

**Варіант 19:** Ускладніть структуру втрат у вашому `SGDClassifier`, додавши «кросентропію з межовим запасом» (margin-based cross-entropy). Це логістична втрата, але ви додаєте невеликий відступ, щоби посилити покарання для прогнозів, близьких до поганого класу. Наприклад, якщо реальна мітка `y=1`, імовірність має бути принаймні 0.7, інакше штрафуйте сильніше, ніж зазвичай. Поставте собі за мету підняти впевненість моделі. Перевірте на `aclImdb_v1.tar.gz`, як зростає час навчання і чи справді це дає приріст до `ROC AUC`. Чи не виникають хибні надмірно впевнені передбачення, які шкодять загальній продуктивності?

*Технічна примітка:* У Python можна модифікувати формулу log loss, додаючи `margin`. Якщо `y=1`, перевіряйте, чи `predict_proba > margin`; і якщо ні, збільшуйте втрату на якийсь коефіцієнт. Для `y=-1` робіть аналогічно, лише зі сторони `1 – predict_proba`. Необхідно чітко задати `margin` і величину штрафу.

---

**Варіант 20:** Удоскональте ваш `SGDClassifier`, застосувавши схему постійної ротації підвибірок даних (sample rotation). Ідея: ви фіксуєте декілька різних підвибірок із `aclImdb_v1.tar.gz` (наприклад, упорядкованих за довжиною тексту, стилем чи іншою характерною ознакою). На першій епосі берете одну підвибірку, на другій – іншу, і так далі, постійно «кружляючи» між цими порціями. Це може стабілізувати оновлення ваг і не давати моделі «застрягти» на специфічних прикладах. Продемонструйте, як змінюється крива `log_loss` і `ROC AUC` при такій ротації та порівняйте з повністю випадковим проходом. Поясніть, у яких задачах обертання підвибірок стає особливо ефективним.

*Технічна примітка:* Підготуйте 3–5 підвибірок за різними критеріями (наприклад, короткі тексти, середні тексти, дуже довгі тексти). Реалізуйте генерацію індексів, чергуючи ці підвибірки, а тоді викликайте оновлення ваг. Використайте `numpy` чи чисті цикли Python для перемішування.